# Ciclo KDD 
## Aplicado a Predicción de Demanda

**Objetivo:** Aplicar el ciclo completo de Knowledge Discovery in Databases (KDD) de forma práctica para construir un modelo predictivo simple de la demanda diaria de croissants en una panadería en base a datos sintéticos.



---

# <div style="text-align: center; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">"El Buen Croissant".</div>
# <div style="text-align: center;"><img src="./aux/medialuna.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>
# <div style="text-align: center; text-shadow: 2px 2px 4px rgba(31, 2, 63, 0.3);">... Quiere saber cómo mejorar sus ventas.</div>

---


## Inmersión en Datos

In [ ]:
# Importaciones principales

# Librerías básicas
import os
import pickle  # Para guardar el DataFrame
import subprocess
import warnings
from collections import Counter

# Análisis de datos
import numpy as np
import pandas as pd
from pandas.io.parquet import to_parquet

# Visualización
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from IPython.display import Image, display

# Estadísticas y modelado
from scipy import stats
import statsmodels.api as sm

# Scikit-learn
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score
)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import (
    DecisionTreeRegressor,
    export_text,
    export_graphviz,
    plot_tree,
    _tree
)

# Otras librerías
import holidays
import shap

# Funciones propias
from funciones.describe_vars import describe_vars


In [73]:
plt.style.use('ggplot') #https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
sns.set_context("notebook", font_scale=1.2)
warnings.filterwarnings('ignore')

### Detalle: Configuración de Visualización y Manejo de Errores

1. **Configuración Visual**: Se pueden definir los estilos y parámetros para las visualizaciones usando seaborn y matplotlib, asegurando una presentación consistente y profesional de nuestros gráficos.

2. **Manejo de Errores**: Se han implementado funciones de utilidad para el manejo de errores y validación de datos, lo que nos permitirá tener un mejor control sobre posibles problemas durante el análisis.

Estas configuraciones van a afectar todo el resto del flujo de trabajo en el archivo. Se pueden agregar más, o especificar luego en cada celda. 

In [74]:
# --- abrimos el csv ---
DATA_FILE = 'panaderia_croissant_synthetic.csv'

# csv a data frame df_raw
df_raw = pd.read_csv(DATA_FILE)


# Es mejor usar la siguiente celda que implementa la carga con PKL o parquet, usamos pkl por ahora. 
# para mejorar la eficiencia en ejecuciones posteriores. Vamos a hacerlo correctamente.

# <div style="text-align: center;"><img src="./aux/kdd_1.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


In [ ]:
# --- Carga de Datos y Guardado en PKL ---
DATA_FILE = 'panaderia_croissant_synthetic.csv'
PKL_FILE = 'croissant_demand_raw.pkl'

try:
    # Intentar cargar PKL primero
    with open(PKL_FILE, 'rb') as f:
        df_raw = pickle.load(f)
    print(f"DataFrame cargado desde '{PKL_FILE}' exitosamente.")
except FileNotFoundError:
    print(f"Archivo PKL no encontrado. Intentando cargar CSV...")
    try:
        # Si no existe PKL, cargar CSV
        df_raw = pd.read_csv(DATA_FILE)
        print(f"Archivo '{DATA_FILE}' cargado exitosamente.")
        
        # Guardar en PKL para futuras ejecuciones
        with open(PKL_FILE, 'wb') as f:
            pickle.dump(df_raw, f)
        print(f"DataFrame guardado en '{PKL_FILE}' para uso futuro.")
        
    except FileNotFoundError:
        print(f"Error: El archivo '{DATA_FILE}' no se encontró.")
        raise SystemExit("No se puede continuar sin datos.")

# Carga con try-except

El bloque try-except es una estructura de control que nos permite manejar posibles errores:

1. El código primero intenta (try) cargar el archivo PKL que es más rápido y eficiente:
   - Si el archivo existe, lo carga y continúa normalmente
   - Si el archivo no existe, Python genera un error FileNotFoundError

2. Cuando ocurre el error, el código salta al bloque except:
   - Aquí tenemos un plan B: intentar cargar el CSV original
   - Usamos otro try-except anidado por si tampoco existe el CSV
   - Si el CSV existe, lo carga y además lo guarda como PKL para la próxima vez
   - Si el CSV tampoco existe, termina el programa con un mensaje de error


---

# Qué vamos a hacer? 


---
# <div style="text-align: center;"><img src="./aux/medialuna.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>
---


#### 1. Carga de datos, manipulación y preprocesamiento

---

#### 2. Transformación de variables (iteración 1)
#### 3. Análisis exploratorio (descriptivas básicas)
#### 2.1 Transformación de variables (iteración 2)
#### 3.1 Análisis exploratorio (correlaciones, regresiones, ANOVA)
#### 3.2 Intuiciones exploratorias

---

#### 4. Definición de predictores (X) y objetivo (y)
#### 5. División temporal Train/Test

---

#### 6. Modelado predictivo: qué modelos para qué problemas
#### 6. Modelado predictivo: entrenamiento, validación, comparación

---

#### 7. Elección del mejor modelo (Supervised Machine Learning with Decision Trees)
#### 8. Despliegue del modelo seleccionado


---

#### 9. Validación del modelo
#### 10. Interpretación del modelo


    ... además... vamos a seguir aprendiendo python, pandas y herramientas/técnicas de análisis 

---

# <div style="text-align: center;"><img src="./aux/kdd_2.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


## Información básica del df


In [ ]:
# --- Carga desde PKL y Descripción Inicial ---
with open(PKL_FILE, 'rb') as f:
    df = pickle.load(f)


# Dimensiones del DataFrame:
# Filas: {df.shape[0]}, Columnas: {df.shape[1]}
df.shape


In [ ]:
# Tipos de datos por columna:
df.dtypes

In [ ]:
df.info()

In [ ]:
# Primeras 3 filas del dataset crudo:
df.head(3)

In [ ]:
df.tail(7)

In [ ]:
df.describe()

# <div style="text-align: center;"><img src="./aux/kdd_3.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# Ajustar el DataFrame a Serie Temporal

Este paso es importante para trabajar con series temporales:

1. Primero, convertiremos la columna 'Fecha' al tipo datetime de pandas
2. Luego, estableceremos esta columna como el índice del DataFrame
3. Finalmente, exploraremos las nuevas capacidades que nos brinda tener un índice temporal:
   - Verificaremos el rango de fechas en nuestros datos
   - Identificaremos el período que abarcan los datos
   - Confirmaremos la frecuencia de las observaciones

Esta transformación nos permitirá:
- Realizar análisis temporales más sofisticados
- Utilizar funcionalidades específicas de series temporales
- Facilitar la visualización de tendencias y patrones a lo largo del tiempo

In [ ]:
print("Tipo de la columna Fecha:", df['Fecha'].dtype)
print("Primeras 3 fechas:", df['Fecha'].head(3).tolist())

In [ ]:
# Guardar una muestra del estado anterior
# Mostramos el tipo de la columna Fecha y las primeras 3 fechas antes de la transformación
display(pd.DataFrame({
    'Tipo de la columna Fecha': [df['Fecha'].dtype],
    'Primeras 3 fechas': [df['Fecha'].head(3).tolist()]
}))


In [84]:

# Convertir a datetime y establecer como índice
df['Fecha'] = pd.to_datetime(df['Fecha'])
df.set_index('Fecha', inplace=True)


In [ ]:
# Información después de la transformación
print("Tipo del índice:", df.index.dtype)
print("Primeras 3 fechas del índice:", df.index[:3].tolist())



In [ ]:
df.head()

# Importancia del uso del Índice Temporal

La transformación realizada, convirtiendo la columna 'Fecha' a tipo datetime y estableciéndola como índice del DataFrame, es fundamental por varias razones:

1. **Funcionalidades Específicas de Series Temporales**:
   - Permite acceder a componentes temporales como año, mes, día, etc.
   - Facilita el cálculo de diferencias temporales y rangos de fechas
   - Habilita el uso de funciones específicas para análisis de series temporales

2. **Mejora en la Manipulación de Datos**:
   - Permite realizar resampling (cambios en la frecuencia de los datos)
   - Facilita la selección y filtrado por períodos específicos

3. **Ventajas para la Visualización**:
   - Los gráficos temporales se generan automáticamente con el eje X correctamente formateado
   - Facilita la identificación de patrones, tendencias y estacionalidad
   - Mejora la interpretación de la evolución temporal de las variables

4. **Validación y Calidad de Datos**:
   - Permite detectar gaps en la serie temporal
   - Facilita la identificación de frecuencias en los datos
   - Ayuda a verificar la consistencia temporal de las observaciones



In [ ]:
# Capacidades adicionales con el índice datetime
print("\nInformación temporal:")
print("Año más antiguo:", df.index.year.min())
print("Año más reciente:", df.index.year.max()) 
print("Rango de fechas:", df.index.max() - df.index.min())
print("Frecuencia de los datos:", pd.infer_freq(df.index))

df

# <div style="text-align: center;"><img src="./aux/kdd_3a.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# Exploración: Análisis descriptivo
 


In [ ]:

# Estadísticas descriptivas básicas
print("\nEstadísticas Descriptivas Básicas:")
print(df.describe().round(2).T)
# round() redondea los valores a 2 decimales para una mejor visualización

# .T transpone el DataFrame, convirtiendo las filas en columnas y viceversa
# Esto facilita la lectura de las estadísticas descriptivas, mostrando cada variable en una fila
# en lugar de en columnas


In [ ]:
#print(df.describe().round(2).T)
display(df.describe().round(2).T)

In [ ]:
# Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# Verificar duplicados
print("\nNúmero de filas duplicadas:", df.duplicated().sum())
print("Número de índices (fechas) duplicados:", df.index.duplicated().sum())

In [ ]:
describe_vars(df)

# Análisis Descriptivo Adicional (sobre t)

En esta sección realizaremos un análisis descriptivo avanzado que incluirá:

- Análisis de ventas por día de la semana
- Análisis de ventas por mes 
- Matriz de correlación entre variables numéricas
- Comparación de ventas con y sin promoción
- Distribución de variables categóricas

In [ ]:

# Estadísticas por día de la semana
print("\nEstadísticas de Cantidad_Vendida por día de la semana:")
display(df.groupby('Dia_Semana')['Cantidad_Vendida'].agg(['count', 'mean', 'std', 'min', 'max']).round(2))

In [ ]:

# Estadísticas por mes
print("\nEstadísticas de Cantidad_Vendida por mes:")
display(df.groupby('Mes')['Cantidad_Vendida'].agg(['count', 'mean', 'std', 'min', 'max']).round(2))

In [ ]:


# Estadísticas para días con y sin promoción
print("\nComparación de ventas con y sin promoción:")
display(df.groupby('Promocion_Croissant')['Cantidad_Vendida'].agg(['count', 'mean', 'std', 'min', 'max']).round(2))


In [ ]:
# Resumen de variables categóricas
print("\nDistribución de variables categóricas:")
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\nDistribución de {col}:")
    display(df[col].value_counts())

In [ ]:
describe_vars(df)

In [ ]:

# Análisis de correlaciones
print("\nMatriz de correlación para variables numéricas:")
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr().round(2)
display(correlation_matrix)


#### Comentarios de "np.number"

In [ ]:

# Análisis de correlaciones
print("\nMatriz de correlación para variables numéricas:")

numeric_cols = df.select_dtypes(include=[np.number]).columns
# select_dtypes() es un método de pandas que filtra columnas según su tipo de dato
# include=[np.number] especifica que solo queremos columnas numéricas (int64, float64)
# .columns devuelve los nombres de las columnas filtradas
# En este caso seleccionará: Mes, Anio, Dia_Anio, Es_Feriado, Temperatura_Max_Prevista,
# Promocion_Croissant, Precio_Nuestro, Precio_Competencia, Cantidad_Vendida
# Ejemplos de select_dtypes() con diferentes tipos:

# Para seleccionar columnas numéricas (como lo usamos arriba):
# df.select_dtypes(include=[np.number]).columns
# Seleccionaría: Mes, Anio, Dia_Anio, Es_Feriado, etc.

# Para seleccionar columnas de texto (strings):
# df.select_dtypes(include=['object']).columns
# Seleccionaría: 'Dia_Semana' (ej: "lunes", "martes")

# Para seleccionar fechas:
# df.select_dtypes(include=['datetime64']).columns 
# Seleccionaría columnas de fecha si 'Fecha' fuera datetime
# Nota: En nuestro caso 'Fecha' es tipo object porque no se convirtió a datetime

# Para seleccionar múltiples tipos a la vez:
# df.select_dtypes(include=['object', 'datetime64', np.number]).columns

# Para excluir ciertos tipos:
# df.select_dtypes(exclude=['object']).columns
# Excluiría 'Dia_Semana' y 'Fecha' (si son object)


correlation_matrix = df[numeric_cols].corr().round(2)

display(correlation_matrix)


In [ ]:
# variantes simples de np.number


int_cols = df.select_dtypes(include=['int64']).columns
display(int_cols)

float_cols = df.select_dtypes(include=['float64']).columns
display(float_cols)

In [ ]:
# Crear el heatmap de correlaciones con fuentes más pequeñas y fondo blanco
plt.figure(figsize=(12, 8))
sns.set_style("whitegrid")  # Aplicar estilo de seaborn
sns.set_context("notebook", font_scale=1.1)  # Ajustar contexto de seaborn

sns.heatmap(correlation_matrix, 
            annot=True,  # Muestra los valores numéricos
            cmap='coolwarm',  # Esquema de colores
            center=0.1,  # Centra el mapa de colores en 0
            fmt='.2f',  # Formato de los números (2 decimales)
            square=True,  # Hace que las celdas sean cuadradas
            cbar_kws={'label': 'Coeficiente de Correlación'},
            annot_kws={'size': 8},  # Tamaño de fuente más pequeño para los valores
            linewidths=0.9)  # Líneas más finas entre celdas

plt.title('Matriz de Correlación', fontsize=12)  # Ajustar tamaño del título
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# respecto de la correlación 1 entre mes y año. 

plt.figure(figsize=(10, 6))
plt.scatter(df['Mes'], df['Dia_Anio'], alpha=0.5)
plt.title('Relación entre Mes y Día del Año')
plt.xlabel('Mes')
plt.ylabel('Día del Año')
plt.grid(True)
plt.show()

# Ahora en v2!

# <div style="text-align: center;"><img src="./aux/kdd_3a.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# Exploración: EDA y Preprocesamiento

---


Objetivo: Entender patrones, relaciones, estacionalidad, tendencia y posibles problemas en los datos. Esto guiará la transformación y el modelado.

---
# Estático y Dinámico

---

### Análisis Estático: Visualizaciones y Distribuciones

1. Distribuciones y Estadísticas Básicas
   - Histogramas y boxplots de la variable objetivo (Cantidad_Vendida)
   - Estadísticas descriptivas: media, mediana, desviación estándar, etc.
   - Identificación de valores atípicos y su posible origen

2. Análisis por Variables Categóricas
   - Ventas promedio por día de la semana (barplot)
   - Ventas mensuales agregadas (barplot)
   - Comparación de ventas en días festivos vs no festivos (boxplot)
   - Impacto de promociones en las ventas (boxplot comparativo)

3. Relaciones entre Variables Numéricas
   - Matriz de correlación con heatmap
   - Scatterplots de variables relevantes vs Cantidad_Vendida
   - Relación entre precios (nuestro vs competencia) y ventas
   - Efecto de la temperatura en las ventas

4. Análisis de Composición
   - Proporción de ventas por temporada
   - Distribución de ventas según rangos de precio
   - Participación por día de la semana
   - Comparación interanual de patrones
   ---


### Preguntas Clave para el Análisis Visual de la Serie Temporal

1. ¿Cuál es el comportamiento general de las ventas a lo largo del tiempo?
   - ¿Existen patrones claros en la serie temporal?
   - ¿Hay valores atípicos o cambios bruscos significativos?
   - ¿Se pueden identificar componentes de estacionalidad o tendencia a simple vista?

2. ¿Cómo varían las tendencias según la escala temporal analizada?
   - ¿Qué patrones emergen al analizar ventanas mensuales (30 días)?
   - ¿Existen ciclos trimestrales (90 días) identificables?
   - ¿Cuál es la tendencia general anual (365 días) de las ventas?
   - ¿Cómo se relacionan los patrones entre diferentes escalas temporales?

3. ¿Cuáles son los componentes fundamentales que explican el comportamiento de las ventas?
   - ¿Qué proporción de la variabilidad se explica por la tendencia?
   - ¿Qué tan fuerte es el componente estacional?
   - ¿Existe un patrón sistemático en los residuos?
   - ¿Hay eventos o factores externos que afecten significativamente la serie?

4. ¿Cómo podemos utilizar estos insights para mejorar nuestro modelo?
   - ¿Qué características deberíamos incluir en el modelo?
   - ¿Qué transformaciones de datos serían más apropiadas?
   - ¿Qué métricas base podemos establecer para evaluar el rendimiento?

   ---
   

#### Conceptos Clave y Técnicas: Visualización Series Temporales


**Series Temporales**
- Secuencia de datos ordenados cronológicamente que captura la evolución de las ventas
- Permite identificar patrones, ciclos y comportamientos a lo largo del tiempo
- Base fundamental para el análisis predictivo de las ventas futuras

**Media Móvil**
- Técnica de suavizado que promedia valores en una ventana deslizante
- Reduce el ruido y resalta tendencias subyacentes
- Ventanas utilizadas:
  * 30 días: patrones mensuales
  * 90 días: patrones trimestrales
  * 365 días: tendencia anual

**Descomposición Estacional**
- Separa la serie temporal en componentes fundamentales:
  * Tendencia: patrón de largo plazo que indica la dirección general
  * Estacionalidad: patrones cíclicos que se repiten en intervalos fijos
  * Residuos: variaciones aleatorias no explicadas por tendencia ni estacionalidad

**Análisis de Ruido y Residuos**
- Ruido: fluctuaciones aleatorias de alta frecuencia en los datos
- Residuos: diferencia entre valores observados y componentes sistemáticos
- Importancia para:
  * Evaluar la calidad del modelo
  * Identificar anomalías
  * Verificar supuestos estadísticos

**Patrones y Ciclos**
- Estacionalidad: variaciones regulares y predecibles
- Ciclos: fluctuaciones no necesariamente regulares
- Tendencias: dirección general del comportamiento a largo plazo

**Valores Atípicos**
- Observaciones que se desvían significativamente del patrón esperado
- Pueden indicar:
  * Eventos especiales
  * Errores en los datos
  * Cambios estructurales en el negocio

Estas técnicas y conceptos nos permitirán:
- Comprender la estructura temporal de las ventas
- Identificar factores que influyen en el comportamiento
- Preparar los datos para el modelado predictivo
- Establecer bases para la evaluación del modelo

---

# Análisis Estático

---

In [ ]:

# Histograma de distribución de ventas
plt.figure(figsize=(10, 5))
df['Cantidad_Vendida'].hist(bins=50, edgecolor='black')
plt.title('Distribución de Ventas Diarias')
plt.xlabel('Cantidad Vendida')
plt.ylabel('Frecuencia')
plt.grid(True, linestyle='--', alpha=0.2)
plt.tight_layout()
plt.show()

In [ ]:

# Frecuencia de ventas diarias promedio en un año
df['DiaDelAño'] = df.index.dayofyear
promedio_anual = df.groupby('DiaDelAño')['Cantidad_Vendida'].mean()

plt.figure(figsize=(10, 5))
promedio_anual.hist(bins=50, edgecolor='black')
plt.title('Distribución de Ventas Diarias (Promedio Anual)')
plt.xlabel('Cantidad Vendida Promedio')
plt.ylabel('Frecuencia')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Frecuencia de ventas diarias promedio en un mes
df['DiaDelMes'] = df.index.day
promedio_mensual = df.groupby('DiaDelMes')['Cantidad_Vendida'].mean()

plt.figure(figsize=(10, 5))
promedio_mensual.hist(bins=15, edgecolor='black')
plt.title('Distribución de Ventas Diarias (Promedio Mensual)')
plt.xlabel('Cantidad Vendida Promedio')
plt.ylabel('Frecuencia')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

Mensual:

	•	El patrón sugiere que hay bastante variabilidad estacional dentro del año.

Diario:	
    
    •	La dispersión es más limitada que en el gráfico anual, lo cual es lógico porque se eliminan las variaciones estacionales más largas.


In [ ]:

# KDE Plot (distribución suavizada)
plt.figure(figsize=(10, 5))
sns.kdeplot(df['Cantidad_Vendida'], shade=True)
plt.title('Densidad Suavizada de Ventas Diarias')
plt.xlabel('Cantidad Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Boxplot global
plt.figure(figsize=(6, 5))
sns.boxplot(y=df['Cantidad_Vendida'])
plt.title('Boxplot Global de Ventas Diarias')
plt.ylabel('Cantidad Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# QQ Plot (normalidad: desviación de la normalidad es desviación de la diagonal)
sm.qqplot(df['Cantidad_Vendida'], line='s')
plt.title('QQ Plot de Ventas Diarias')
plt.tight_layout()
plt.show()

# Boxplot por día de la semana
df['DiaSemana'] = df.index.dayofweek
plt.figure(figsize=(10, 5))
sns.boxplot(x='DiaSemana', y='Cantidad_Vendida', data=df)
plt.title('Boxplot por Día de la Semana (0=Lunes, 6=Domingo)')
plt.xlabel('Día de la Semana')
plt.ylabel('Cantidad Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# Boxplot por mes
df['Mes'] = df.index.month
plt.figure(figsize=(10, 5))
sns.boxplot(x='Mes', y='Cantidad_Vendida', data=df)
plt.title('Boxplot por Mes ')
plt.xlabel('Mes')
plt.ylabel('Cantidad Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# Promedio por mes del año
df['Mes'] = df.index.month
prom_mensual = df.groupby('Mes')['Cantidad_Vendida'].mean()
plt.figure(figsize=(10, 5))
prom_mensual.plot(kind='bar')
plt.title('Promedio de Ventas por Mes del Año')
plt.xlabel('Mes')
plt.ylabel('Cantidad Promedio Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Promedio por semana del año
df['Semana'] = df.index.isocalendar().week
prom_semanal = df.groupby('Semana')['Cantidad_Vendida'].mean()
plt.figure(figsize=(12, 5))
prom_semanal.plot()
plt.title('Promedio de Ventas por Semana del Año')
plt.xlabel('Semana')
plt.ylabel('Cantidad Promedio Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()




In [ ]:

# Preparar columnas derivadas
df['DiaDelAnio'] = df.index.dayofyear
df['DiaDelMes'] = df.index.day
df['DiaSemana'] = df.index.dayofweek
df['Mes'] = df.index.month
df['Anio'] = df.index.year


# Serie completa
sns.kdeplot(df['Cantidad_Vendida'], label='Global', linewidth=2)

# Promedio por día del año (acumulado entre años)
prom_dia_anio = df.groupby('DiaDelAnio')['Cantidad_Vendida'].mean()
sns.kdeplot(prom_dia_anio, label='Promedio por Día del Año')

# Promedio por mes
prom_mes = df.groupby('Mes')['Cantidad_Vendida'].mean()
sns.kdeplot(prom_mes, label='Promedio por Mes')

# Promedio por día del mes
prom_dia_mes = df.groupby('DiaDelMes')['Cantidad_Vendida'].mean()
sns.kdeplot(prom_dia_mes, label='Promedio por Día del Mes')

# Promedio por día de la semana
prom_semana = df.groupby('DiaSemana')['Cantidad_Vendida'].mean()
sns.kdeplot(prom_semana, label='Promedio por Día de Semana')

plt.title('KDE Comparativo: Niveles Agregados sin Tiempo Explícito')
plt.xlabel('Cantidad Vendida Promedio')
plt.ylabel('Densidad')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()


########################∫
# Mismo gráfico sin el promedio por día del año
plt.figure()
sns.kdeplot(df['Cantidad_Vendida'], label='Global', linewidth=2)

# Promedio por mes
sns.kdeplot(prom_mes, label='Promedio por Mes')

# Promedio por día del año
sns.kdeplot(prom_dia_anio, label='Promedio por Día del Año')

# Promedio por día de la semana
sns.kdeplot(prom_semana, label='Promedio por Día de Semana')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# Clasificación por quintil
df['Quintil'] = pd.qcut(df['Cantidad_Vendida'], 5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])




# Violin plot por quintil
plt.figure(figsize=(8, 5))
sns.violinplot(x='Quintil', y='Cantidad_Vendida', data=df, order=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
plt.title('Distribución de Ventas por Quintil (Violin Plot)')
plt.xlabel('Quintil')
plt.ylabel('Cantidad Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()



# KDE por quintil
plt.figure(figsize=(10, 5))
for quintil in ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']:
    sns.kdeplot(df[df['Quintil'] == quintil]['Cantidad_Vendida'], label=quintil)

plt.title('Distribución KDE por Quintil de Ventas')
plt.xlabel('Cantidad Vendida')
plt.ylabel('Densidad')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()



#### Vuelta 3!!! - Datos de series de tiempo

# <div style="text-align: center;"><img src="./aux/kdd_3a.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


---

# Análisis Dinámico 


---


In [ ]:
# Visualizar la serie de tiempo completa
plt.figure(figsize=(16, 6))
df['Cantidad_Vendida'].plot()
plt.title('Ventas Diarias de Croissants de Almendras (5 Años)')
plt.ylabel('Cantidad Vendida')
plt.xlabel('Fecha')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()




In [ ]:

# Media móvil de 7 días
df['MediaMovil7'] = df['Cantidad_Vendida'].rolling(window=7).mean()
plt.figure(figsize=(12, 5))
df['Cantidad_Vendida'].plot(alpha=0.3, label='Original')
df['MediaMovil7'].plot(label='Media Móvil (7 días)', linewidth=2)
plt.title('Media Móvil de Ventas Diarias (7 días)')
plt.xlabel('Fecha')
plt.ylabel('Cantidad Vendida')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# Media móvil de 7 días
df['MediaMovil30'] = df['Cantidad_Vendida'].rolling(window=30).mean()
plt.figure(figsize=(12, 5))
df['Cantidad_Vendida'].plot(alpha=0.3, label='Original')
df['MediaMovil30'].plot(label='Media Móvil (30 días)', linewidth=2)
plt.title('Media Móvil de Ventas Diarias (30 días)')
plt.xlabel('Fecha')
plt.ylabel('Cantidad Vendida')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# Scatter: ventas vs día del año
df['DiaDelAño'] = df.index.dayofyear
plt.figure(figsize=(10, 5))
plt.scatter(df['DiaDelAño'], df['Cantidad_Vendida'], alpha=0.3)
plt.title('Ventas vs Día del Año')
plt.xlabel('Día del Año')
plt.ylabel('Cantidad Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Scatter: ventas vs día del mes
df['DiaDelMes'] = df.index.day
plt.figure(figsize=(10, 5))
plt.scatter(df['DiaDelMes'], df['Cantidad_Vendida'], alpha=0.3)
plt.title('Ventas vs Día del Mes')
plt.xlabel('Día del Mes')
plt.ylabel('Cantidad Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:

# Boxplot por año
plt.figure(figsize=(12, 6))
df['Año'] = df.index.year
df.boxplot(column='Cantidad_Vendida', by='Año')
plt.title('Distribución Anual de Ventas')
plt.suptitle('')
plt.xlabel('Año')
plt.ylabel('Cantidad Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Promedio mensual (para ver estacionalidad en agregados)
df['Mes'] = df.index.month
promedios_mensuales = df.groupby('Mes')['Cantidad_Vendida'].mean()

plt.figure(figsize=(10, 5))
promedios_mensuales.plot(marker='o')
plt.title('Promedio de Ventas por Mes (Agregado Anual)')
plt.xlabel('Mes')
plt.ylabel('Cantidad Promedio Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(range(1, 13))
plt.tight_layout()
plt.show()



# Día del mes
df['DiaDelMes'] = df.index.day

# Promedio por día del mes (agregado en todos los meses)
promedio_diario_mes = df.groupby('DiaDelMes')['Cantidad_Vendida'].mean()

# Gráfico del promedio de ventas por día del mes
plt.figure(figsize=(12, 5))
promedio_diario_mes.plot(marker='o')
plt.title('Promedio de Ventas por Día del Mes (Agregado Anual)')
plt.xlabel('Día del Mes')
plt.ylabel('Cantidad Promedio Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(range(1, 32))
plt.tight_layout()
plt.show()

In [ ]:
# Calcular y graficar media móvil para suavizar y ver tendencias/ciclos
plt.figure(figsize=(16, 6))
df['Cantidad_Vendida'].plot(label='Diario', alpha=0.6, style='.') # Usar puntos para ver mejor densidad
df['Cantidad_Vendida'].rolling(window=30).mean().plot(label='Media Móvil 30 días', linewidth=2)
df['Cantidad_Vendida'].rolling(window=90).mean().plot(label='Media Móvil 90 días', linewidth=2, linestyle='--')
df['Cantidad_Vendida'].rolling(window=365).mean().plot(label='Media Móvil 365 días', linewidth=3, color='red')
plt.title('Ventas Diarias y Medias Móviles')
plt.ylabel('Cantidad Vendida')
plt.xlabel('Fecha')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()



In [ ]:
# Asegurarse que la serie tenga frecuencia diaria
ts_data = df['Cantidad_Vendida'].asfreq('D') # Rellenará con NaN si faltan fechas

# La frecuencia diaria es necesaria para la descomposición estacional porque:
# 1. Implica el mayor nivel de detalle posible. Son los saltos "más chicos" en la serie. 
# 1a. Asegura que no haya "huecos" en la serie temporal que distorsionen el análisis
# 2. Permite identificar correctamente los patrones estacionales al tener observaciones equidistantes
# 3. Es requerido por el método seasonal_decompose que espera datos con frecuencia regular

ts_data.fillna(method='ffill', inplace=True) # Rellenar NaNs si los hubiera

	# •	Quiero descomponer una serie de tiempo (ts_data)
	# •	Bajo la suposición de que los componentes se suman entre sí (model='additive')
	# •	Y que el patrón estacional se repite cada 365 días (period=365), es decir, un ciclo anual


# El period=365 indica que buscamos patrones anuales, pero podríamos usar:
# - period=7 para patrones semanales
# - period=30/31 para patrones mensuales
# - otros valores según el ciclo estacional que queramos analizar

# sm.tsa.seasonal_decompose() descompone una serie temporal en 3 componentes:
# - Tendencia: dirección a largo plazo de la serie
# - Estacionalidad: patrones cíclicos que se repiten con frecuencia fija
# - Residuos: variaciones aleatorias no explicadas por tendencia ni estacionalidad
# model='additive' asume que los componentes se suman (vs multiplicativo donde se multiplican)



decomposition = sm.tsa.seasonal_decompose(ts_data, model='additive', period=365)
fig = decomposition.plot()
fig.set_size_inches(14, 10)
plt.suptitle('Descomposición Estacional Aditiva (Ciclo Anual)', y=1.01)
# Rotar etiquetas del eje x para mejor legibilidad
for ax in fig.axes:
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout(rect=[0, 0.03, 1, 0.99])
plt.show()

In [ ]:
# El mismo gráfico de arriba, pero con los componentes separados

# Gráfico: Tendencia
plt.figure(figsize=(14, 4))
plt.plot(decomposition.trend)
plt.title('Componente: Tendencia')
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico: Estacionalidad
plt.figure(figsize=(14, 4))
plt.plot(decomposition.seasonal)
plt.title('Componente: Estacionalidad')
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico: Residuos
plt.figure(figsize=(14, 4))
plt.plot(decomposition.resid)
plt.title('Componente: Residuos')
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Interpretación EDA - Variable Objetivo:**
*   **Tendencia:** ¿La media móvil de 365 días muestra una tendencia general (creciente, decreciente, estable)? (Esperamos un ligero crecimiento por la simulación).
*   **Estacionalidad Anual:** ¿Se observa claramente el ciclo en la media móvil de 30/90 días y en la componente `seasonal` de la descomposición? (Ventas más altas en invierno, más bajas en verano).
*   **Eventos:** ¿Hay picos o valles abruptos en la serie diaria? ¿Coinciden visualmente con feriados o posibles promociones?
*   **Ruido/Residuos:** La componente `resid` muestra la variabilidad no explicada por la tendencia y la estacionalidad simple. ¿Parece aleatoria o tiene algún patrón?

# <div style="text-align: center;"><img src="./aux/kdd_4.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# Transformación: categóricas y binarias

#### 2.1.2. Influencia de Variables Categóricas / Binarias

In [ ]:
# Crear mapeo de días a números
dia_a_numero = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'domingo': 6}
df['Dia_Semana_Num'] = df['Dia_Semana'].map(dia_a_numero)

# Crear el gráfico
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Distribución de Ventas por Categorías', fontsize=16)

# Ordenar días de la semana correctamente
weekday_order = list(range(7))  # 0 a 6
sns.boxplot(ax=axes[0, 0], data=df, x='Dia_Semana_Num', y='Cantidad_Vendida', order=weekday_order)
axes[0, 0].set_title('Ventas por Día de la Semana')
axes[0, 0].set_xticklabels(['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'], rotation=30)
axes[0, 0].set_xlabel('')
axes[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

sns.boxplot(ax=axes[0, 1], data=df, x='Mes', y='Cantidad_Vendida')
axes[0, 1].set_title('Ventas por Mes')
axes[0, 1].set_xlabel('Mes del Año')
axes[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

sns.boxplot(ax=axes[1, 0], data=df, x='Es_Feriado', y='Cantidad_Vendida')
axes[1, 0].set_title('Ventas en Días Feriados/Puente vs. Normales')
axes[1, 0].set_xticklabels(['Normal (0)', 'Feriado/Puente (1)'])
axes[1, 0].set_xlabel('')
axes[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

sns.boxplot(ax=axes[1, 1], data=df, x='Promocion_Croissant', y='Cantidad_Vendida')
axes[1, 1].set_title('Ventas con Promoción vs. Sin Promoción')
axes[1, 1].set_xticklabels(['Sin Promoción (0)', 'Con Promoción (1)'])
axes[1, 1].set_xlabel('')
axes[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.96]) # Ajustar para el titulo gral
plt.show()

In [ ]:
# Crear indicador de fin de semana
df['Es_Fin_Semana'] = df['Dia_Semana'].isin(['sábado', 'domingo']).astype(int)

# Frecuencias relativas generales - subgráficos
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

# Obtener colores del estilo actual
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Primer subgráfico - Feriados
df['Es_Feriado'].value_counts(normalize=True).plot(kind='bar', ax=ax1, color=[colors[0], colors[1]])
ax1.set_title('Frecuencia Relativa:\nDías Normales vs Feriados (Total)')
ax1.set_xlabel('Tipo de Día')
ax1.set_ylabel('Frecuencia Relativa')
ax1.grid(axis='y', linestyle='--', alpha=0.7)
ax1.set_xticklabels(['Días Normales', 'Días Feriados'], rotation=0)
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Segundo subgráfico - Promociones
df['Promocion_Croissant'].value_counts(normalize=True).plot(kind='bar', ax=ax2, color=[colors[0], colors[1]])
ax2.set_title('Frecuencia Relativa:\nSin vs Con Promoción (Total)')
ax2.set_xlabel('Promoción')
ax2.set_ylabel('Frecuencia Relativa')
ax2.grid(axis='y', linestyle='--', alpha=0.7)
ax2.set_xticklabels(['Sin Promoción', 'Con Promoción'], rotation=0)
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Tercer subgráfico - Fines de semana
df['Es_Fin_Semana'].value_counts(normalize=True).plot(kind='bar', ax=ax3, color=[colors[0], colors[1]])
ax3.set_title('Frecuencia Relativa:\nDías Laborales vs Fin de Semana (Total)')
ax3.set_xlabel('Tipo de Día')
ax3.set_ylabel('Frecuencia Relativa')
ax3.grid(axis='y', linestyle='--', alpha=0.7)
ax3.set_xticklabels(['Días Laborales', 'Fin de Semana'], rotation=0)
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

# Por año
plt.figure(figsize=(12, 6))
df.groupby(['Anio', 'Es_Feriado']).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar')
plt.title('Frecuencia Relativa: Días Normales vs Feriados por Año')
plt.xlabel('Año')
plt.ylabel('Frecuencia Relativa')
plt.legend(['Días Normales', 'Días Feriados'], bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
df.groupby(['Anio', 'Promocion_Croissant']).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar')
plt.title('Frecuencia Relativa: Sin vs Con Promoción por Año')
plt.xlabel('Año')
plt.ylabel('Frecuencia Relativa')
plt.legend(['Sin Promoción', 'Con Promoción'], bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
df.groupby(['Anio', 'Es_Fin_Semana']).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar')
plt.title('Frecuencia Relativa: Días Laborales vs Fin de Semana por Año')
plt.xlabel('Año')
plt.ylabel('Frecuencia Relativa')
plt.legend(['Días Laborales', 'Fin de Semana'], bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Por mes
plt.figure(figsize=(14, 6))
df.groupby(['Mes', 'Es_Feriado']).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar')
plt.title('Frecuencia Relativa: Días Normales vs Feriados por Mes')
plt.xlabel('Mes')
plt.ylabel('Frecuencia Relativa')
plt.legend(['Días Normales', 'Días Feriados'], bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

plt.figure(figsize=(14, 6))
df.groupby(['Mes', 'Promocion_Croissant']).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar')
plt.title('Frecuencia Relativa: Sin vs Con Promoción por Mes')
plt.xlabel('Mes')
plt.ylabel('Frecuencia Relativa')
plt.legend(['Sin Promoción', 'Con Promoción'], bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

plt.figure(figsize=(14, 6))
df.groupby(['Mes', 'Es_Fin_Semana']).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar')
plt.title('Frecuencia Relativa: Días Laborales vs Fin de Semana por Mes')
plt.xlabel('Mes')
plt.ylabel('Frecuencia Relativa')
plt.legend(['Días Laborales', 'Fin de Semana'], bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Crear combinaciones de categorías
df['Categoria'] = 'Día Normal'  # categoría por defecto

# Actualizar categorías basadas en combinaciones
df.loc[df['Es_Fin_Semana'] == 1, 'Categoria'] = 'Fin de Semana'
df.loc[df['Es_Feriado'] == 1, 'Categoria'] = 'Feriado'
df.loc[df['Promocion_Croissant'] == 1, 'Categoria'] = 'Promoción'

# Crear combinaciones detalladas
df['Categoria_Detallada'] = 'Día Normal'
df.loc[df['Es_Fin_Semana'] == 1, 'Categoria_Detallada'] = 'Fin de Semana'
df.loc[df['Es_Feriado'] == 1, 'Categoria_Detallada'] = 'Feriado'
df.loc[(df['Es_Fin_Semana'] == 1) & (df['Es_Feriado'] == 1), 'Categoria_Detallada'] = 'Feriado en Fin de Semana'
df.loc[df['Promocion_Croissant'] == 1, 'Categoria_Detallada'] = 'Promoción'
df.loc[(df['Promocion_Croissant'] == 1) & (df['Es_Fin_Semana'] == 1), 'Categoria_Detallada'] = 'Promoción en Fin de Semana'
df.loc[(df['Promocion_Croissant'] == 1) & (df['Es_Feriado'] == 1), 'Categoria_Detallada'] = 'Promoción en Feriado'
df.loc[(df['Promocion_Croissant'] == 1) & (df['Es_Feriado'] == 1) & (df['Es_Fin_Semana'] == 1), 'Categoria_Detallada'] = 'Promoción en Feriado y Fin de Semana'

# Gráfico 1: Distribución general
plt.figure(figsize=(15, 6))
cat_counts = df['Categoria_Detallada'].value_counts()
cat_freq = (cat_counts / len(df) * 100).round(2)
bars = plt.bar(range(len(cat_freq)), cat_freq)
plt.title('Distribución de Días por Categoría (%)')
plt.xlabel('Categoría')
plt.ylabel('Porcentaje')
plt.xticks(range(len(cat_freq)), cat_freq.index, rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Añadir etiquetas de porcentaje sobre las barras
for i, v in enumerate(cat_freq):
    plt.text(i, v + 0.5, f'{v:.1f}%', ha='center')

plt.tight_layout()
plt.show()

# Gráfico 2: Distribución de ventas por categoría
plt.figure(figsize=(15, 6))
sns.boxplot(data=df, x='Categoria_Detallada', y='Cantidad_Vendida')
plt.title('Distribución de Ventas por Categoría')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Gráfico 3: Ventas promedio por categoría
plt.figure(figsize=(15, 6))
mean_sales = df.groupby('Categoria_Detallada')['Cantidad_Vendida'].mean().sort_values(ascending=False)
bars = plt.bar(range(len(mean_sales)), mean_sales)
plt.title('Promedio de Ventas por Categoría')
plt.xlabel('Categoría')
plt.ylabel('Promedio de Ventas')
plt.xticks(range(len(mean_sales)), mean_sales.index, rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Añadir valores promedio sobre las barras
for i, v in enumerate(mean_sales):
    plt.text(i, v + 0.5, f'{v:.1f}', ha='center')

plt.tight_layout()
plt.show()

# Gráfico 4: Gráfico KDE comparando distribuciones
plt.figure(figsize=(15, 6))
for categoria in df['Categoria_Detallada'].unique():
    sns.kdeplot(data=df[df['Categoria_Detallada'] == categoria]['Cantidad_Vendida'], 
                label=categoria)
plt.title('Distribución de Ventas por Categoría (KDE)')
plt.xlabel('Cantidad Vendida')
plt.ylabel('Densidad')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Gráfico 5: Patrones mensuales
plt.figure(figsize=(15, 6))
monthly_avg = df.groupby(['Mes', 'Categoria_Detallada'])['Cantidad_Vendida'].mean().unstack()
monthly_avg.plot(marker='o')
plt.title('Promedio de Ventas por Categoría - Evolución Mensual')
plt.xlabel('Mes')
plt.ylabel('Promedio de Ventas')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Creamos dos visualizaciones para analizar patrones en las ventas según diferentes categorías de días

# Primera visualización: Gráfico de dispersión que muestra los datos individuales
# Creamos una figura de tamaño grande para mejor visualización
plt.figure(figsize=(16, 8))

# Graficamos los días normales (sin feriados, sin fin de semana, sin promociones)
# Usamos puntos pequeños y transparentes para evitar saturación visual
normal_days = df[(df['Es_Feriado'] == 0) & (df['Es_Fin_Semana'] == 0) & (df['Promocion_Croissant'] == 0)]
plt.scatter(normal_days.index.dayofyear, normal_days['Cantidad_Vendida'], 
    alpha=0.3, color='gray', label='Días Normales', s=30)

# Graficamos los feriados con puntos rojos más grandes
holidays = df[df['Es_Feriado'] == 1]
plt.scatter(holidays.index.dayofyear, holidays['Cantidad_Vendida'], 
    alpha=0.6, color='red', label='Feriados', s=50)

# Graficamos los fines de semana sin promoción en azul
weekends = df[(df['Es_Fin_Semana'] == 1) & (df['Promocion_Croissant'] == 0) & (df['Es_Feriado'] == 0)]
plt.scatter(weekends.index.dayofyear, weekends['Cantidad_Vendida'], 
    alpha=0.6, color='blue', label='Fines de Semana', s=50)

# Graficamos los días con promoción (que no son fin de semana) en verde
promos = df[(df['Promocion_Croissant'] == 1) & (df['Es_Fin_Semana'] == 0) & (df['Es_Feriado'] == 0)]
plt.scatter(promos.index.dayofyear, promos['Cantidad_Vendida'], 
    alpha=0.6, color='green', label='Promociones', s=50)

# Graficamos los fines de semana con promoción en morado
weekend_promos = df[(df['Es_Fin_Semana'] == 1) & (df['Promocion_Croissant'] == 1)]
plt.scatter(weekend_promos.index.dayofyear, weekend_promos['Cantidad_Vendida'], 
    alpha=0.6, color='purple', label='Fines de Semana + Promociones', s=50)

# Configuramos los elementos del gráfico
plt.title('Ventas por Día del Año según Categoría')
plt.xlabel('Día del Año')
plt.ylabel('Cantidad Vendida')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Segunda visualización: Media móvil para ver tendencias más claras
# Usamos una ventana de 30 días para suavizar las fluctuaciones diarias
plt.figure(figsize=(16, 8))

# Definimos el tamaño de la ventana para la media móvil
window = 30  # ventana de 30 días

# Calculamos y graficamos la media móvil para cada categoría
# Días normales
normal_rolling = normal_days['Cantidad_Vendida'].rolling(window=window, min_periods=1).mean()
plt.plot(normal_days.index.dayofyear, normal_rolling, 
    color='gray', label='Días Normales', alpha=0.8)

# Feriados
holiday_rolling = holidays['Cantidad_Vendida'].rolling(window=window, min_periods=1).mean()
plt.plot(holidays.index.dayofyear, holiday_rolling, 
    color='red', label='Feriados', alpha=0.8)

# Fines de semana
weekend_rolling = weekends['Cantidad_Vendida'].rolling(window=window, min_periods=1).mean()
plt.plot(weekends.index.dayofyear, weekend_rolling, 
    color='blue', label='Fines de Semana', alpha=0.8)

# Días con promoción
promo_rolling = promos['Cantidad_Vendida'].rolling(window=window, min_periods=1).mean()
plt.plot(promos.index.dayofyear, promo_rolling, 
    color='green', label='Promociones', alpha=0.8)

# Fines de semana con promoción
weekend_promo_rolling = weekend_promos['Cantidad_Vendida'].rolling(window=window, min_periods=1).mean()
plt.plot(weekend_promos.index.dayofyear, weekend_promo_rolling, 
    color='purple', label='Fines de Semana + Promociones', alpha=0.8)

# Configuramos los elementos del gráfico de medias móviles
plt.title(f'Media Móvil ({window} días) de Ventas por Categoría')
plt.xlabel('Día del Año')
plt.ylabel('Cantidad Vendida (Media Móvil)')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# 1. Análisis de tendencia temporal
plt.figure(figsize=(15, 6))
df['Fecha'] = pd.to_datetime(df[['Anio', 'Mes', 'DiaDelMes']].rename(
    columns={'Anio': 'year', 'Mes': 'month', 'DiaDelMes': 'day'}))
df.set_index('Fecha', inplace=True)

# Gráfico de serie temporal con media móvil
plt.plot(df['Cantidad_Vendida'], alpha=0.3, label='Ventas Diarias')
plt.plot(df['MediaMovil7'], label='Media Móvil 7 días', linewidth=2)
plt.title('Evolución de Ventas con Media Móvil')
plt.legend()
plt.show()

# 2. Comparación de ventas por categoría
plt.figure(figsize=(12, 6))
sns.boxplot(x='Categoria', y='Cantidad_Vendida', data=df,
           order=['Día Normal', 'Fin de Semana', 'Feriado', 'Promoción'])
plt.title('Distribución de Ventas por Categoría')
plt.xticks(rotation=45)
plt.show()

# 3. Análisis de ventas por quintil
plt.figure(figsize=(12, 6))
sns.boxplot(x='Quintil', y='Cantidad_Vendida', data=df)
plt.title('Distribución de Ventas por Quintil')
plt.show()

# 4. Comparación de ventas en días normales vs. especiales
plt.figure(figsize=(12, 6))
sns.boxplot(x='Es_Fin_Semana', y='Cantidad_Vendida', data=df)
plt.title('Comparación de Ventas: Días de Semana vs. Fin de Semana')
plt.xticks([0, 1], ['Días de Semana', 'Fin de Semana'])
plt.show()

# 5. Análisis de ventas por mes y año
plt.figure(figsize=(15, 6))
sns.boxplot(x='Mes', y='Cantidad_Vendida', hue='Anio', data=df)
plt.title('Distribución de Ventas por Mes y Año')
plt.legend(title='Año')
plt.show()

# 6. Análisis de ventas acumuladas
plt.figure(figsize=(15, 6))
df['Acumulado_Anual'] = df.groupby('Anio')['Cantidad_Vendida'].cumsum()
sns.lineplot(x='Dia_Anio', y='Acumulado_Anual', hue='Anio', data=df)
plt.title('Ventas Acumuladas por Año')
plt.xlabel('Día del Año')
plt.ylabel('Ventas Acumuladas')
plt.show()

# 7. Comparación de ventas en días con/sin promoción
plt.figure(figsize=(12, 6))
sns.boxplot(x='Promocion_Croissant', y='Cantidad_Vendida', 
           hue='Es_Fin_Semana', data=df)
plt.title('Efecto de Promociones en Días de Semana y Fin de Semana')
plt.xticks([0, 1], ['Sin Promoción', 'Con Promoción'])
plt.legend(title='Fin de Semana', labels=['No', 'Sí'])
plt.show()

# 8. Análisis de ventas por categoría detallada
plt.figure(figsize=(12, 6))
sns.boxplot(x='Categoria_Detallada', y='Cantidad_Vendida', data=df)
plt.title('Distribución de Ventas por Categoría Detallada')
plt.xticks(rotation=45)
plt.show()

## Vuelta 4!!, de nuevo exploramos

# <div style="text-align: center;"><img src="./aux/kdd_3a.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# <div style="text-align: center;"><img src="./aux/kdd_4.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# <div style="text-align: center;"><img src="./aux/kdd_3a.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# <div style="text-align: center;"><img src="./aux/kdd_4.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


In [ ]:
# Ventas vs Temperatura Máxima
plt.figure(figsize=(12, 5))
sns.scatterplot(data=df, x='Temperatura_Max_Prevista', y='Cantidad_Vendida', alpha=0.2, s=100, color='blue')
plt.title('Relación con el entorno - Ventas vs. Temperatura Máx.')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Ventas vs Precio Nuestro
plt.figure(figsize=(12, 5))
sns.scatterplot(data=df, x='Precio_Nuestro', y='Cantidad_Vendida', alpha=0.2, s=100, color='green')
plt.title('Relación con el entorno - Ventas vs. Precio Nuestro')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Ventas vs Precio Competencia
plt.figure(figsize=(12, 5))
sns.scatterplot(data=df, x='Precio_Competencia', y='Cantidad_Vendida', alpha=0.2, s=100, color='purple')
plt.title('Relación con el entorno - Ventas vs. Precio Competencia')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# EDA - Variables categóricas con tratamiento

**EDA - Variables Categóricas clave:**
*   **Día Semana:** ¿Se confirma el aumento de ventas (mediana y dispersión) en fines de semana (Sábado/Domingo)?
*   **Mes:** ¿El patrón estacional visto antes se confirma aquí? (Meses de invierno con medianas más altas).
*   **Feriado:** ¿Hay una diferencia clara y positiva en las ventas durante feriados/puentes?
*   **Promoción:** ¿Las promociones tienen un efecto positivo y significativo en las ventas?


---


In [ ]:
#### aquí la diferencia no son los gráficos, sino el análisis y los indicadores estadísticos que vamos a calcular. 

# 1. Análisis por Día de la Semana
print("=== ANÁLISIS POR DÍA DE LA SEMANA ===")
ventas_dia = df.groupby('Dia_Semana')['Cantidad_Vendida'].agg(['median', 'mean', 'std', 'count'])
print(ventas_dia)

# 2. Análisis por Mes
print("\n=== ANÁLISIS POR MES ===")
ventas_mes = df.groupby('Mes')['Cantidad_Vendida'].agg(['median', 'mean', 'std', 'count'])
print(ventas_mes)

# 3. Análisis por Feriado
print("\n=== ANÁLISIS POR FERIADO ===")
ventas_feriado = df.groupby('Es_Feriado')['Cantidad_Vendida'].agg(['median', 'mean', 'std', 'count'])
print(ventas_feriado)

# 4. Análisis por Promoción
print("\n=== ANÁLISIS POR PROMOCIÓN ===")
ventas_promocion = df.groupby('Promocion_Croissant')['Cantidad_Vendida'].agg(['median', 'mean', 'std', 'count'])
print(ventas_promocion)

# 5. Análisis de significancia estadística

# Prueba t para feriados
feriado_si = df[df['Es_Feriado'] == 1]['Cantidad_Vendida']
feriado_no = df[df['Es_Feriado'] == 0]['Cantidad_Vendida']
t_stat, p_val = stats.ttest_ind(feriado_si, feriado_no, equal_var=False)
print(f"\nPrueba t para feriados: t = {t_stat:.2f}, p-valor = {p_val:.4f}")

# Prueba t para promociones
promo_si = df[df['Promocion_Croissant'] == 1]['Cantidad_Vendida']
promo_no = df[df['Promocion_Croissant'] == 0]['Cantidad_Vendida']
t_stat, p_val = stats.ttest_ind(promo_si, promo_no, equal_var=False)
print(f"Prueba t para promociones: t = {t_stat:.2f}, p-valor = {p_val:.4f}")

# Análisis de varianza (ANOVA) para días de la semana
f_stat, p_val = stats.f_oneway(*[df[df['Dia_Semana'] == dia]['Cantidad_Vendida'] for dia in df['Dia_Semana'].unique()])
print(f"ANOVA para días de la semana: F = {f_stat:.2f}, p-valor = {p_val:.4f}")

# Análisis de varianza (ANOVA) para meses
f_stat, p_val = stats.f_oneway(*[df[df['Mes'] == mes]['Cantidad_Vendida'] for mes in df['Mes'].unique()])
print(f"ANOVA para meses: F = {f_stat:.2f}, p-valor = {p_val:.4f}")

# EDA - Modelización exploratoria

--- 

## Anova 





### Vamos a ejecutar un ANOVA para Días de la Semana

#### ¿Qué es el ANOVA?

ANOVA (Análisis de Varianza) es una herramienta estadística que nos ayuda a responder la pregunta: 
**"¿Las ventas son diferentes según el día de la semana?"**

#### ¿Cómo funciona?

1. **Compara dos tipos de variación**:
   - **Variación entre grupos**: ¿Cuánto difieren las ventas entre lunes, martes, miércoles, etc.?
   - **Variación dentro de grupos**: ¿Cuánto varían las ventas dentro de un mismo día?

2. **El valor F (111.56)** nos dice:
   - Si F es grande (como en este caso), significa que las diferencias entre días son mucho más importantes que las variaciones dentro de cada día


# 3. La fórmula matemática del ANOVA:

F = (Varianza entre grupos) / (Varianza dentro de grupos)

F = (MSB) / (MSW)
    
Donde:
- MSB (Mean Square Between) = SSB / (k-1)
- MSW (Mean Square Within) = SSW / (N-k)
- SSB = Σ nᵢ(x̄ᵢ - x̄)²  (Suma de cuadrados entre grupos)
- SSW = Σ(xᵢⱼ - x̄ᵢ)²   (Suma de cuadrados dentro de grupos)
- k = número de grupos
- N = número total de observaciones
- nᵢ = número de observaciones en el grupo i
- x̄ᵢ = media del grupo i
- x̄ = media general


#### ¿Por qué es útil?

**Identifica patrones importantes**:
   - Nos confirma que no todos los días son iguales
   - Nos dice que el día de la semana es un factor importante para predecir ventas


#### Ejemplo práctico

- **Lunes**: Ventas promedio de 100 unidades
- **Viernes**: Ventas promedio de 300 unidades
- **Sábado**: Ventas promedio de 400 unidades

El ANOVA nos dice que estas diferencias son reales y no producto del azar. Esto significa que:
- No es coincidencia que los sábados vendas más
- Deberías tener más personal los fines de semana
- Deberías preparar más inventario para los días de mayor venta

#### ¿Por qué el p-valor es importante?

- **p-valor = 0.0000** significa que hay menos de 0.01% de probabilidad de que estas diferencias sean producto del azar
- Es como decir: "Estamos 99.99% seguros de que los días de la semana afectan las ventas"



In [ ]:
# Filtrar solo días de semana (lunes a viernes) usando números
df_semana = df[df['Dia_Semana_Num'].between(1, 5)]

# Mapear números a nombres para el gráfico
dias_nombres = {1: 'lunes', 2: 'martes', 3: 'miércoles', 4: 'jueves', 5: 'viernes'}
df_semana['Dia_Nombre'] = df_semana['Dia_Semana_Num'].map(dias_nombres)

# Análisis descriptivo
print("=== ANÁLISIS POR DÍA DE LA SEMANA (LUNES A VIERNES) ===")
ventas_dia_semana = df_semana.groupby('Dia_Nombre')['Cantidad_Vendida'].agg(['median', 'mean', 'std', 'count'])
print(ventas_dia_semana)

# Gráfico de ventas por día
plt.figure(figsize=(10, 6))
sns.boxplot(x='Dia_Nombre', y='Cantidad_Vendida', data=df_semana, 
            order=['lunes', 'martes', 'miércoles', 'jueves', 'viernes'])
plt.title('Distribución de Ventas por Día de la Semana (Lunes a Viernes)')
plt.xticks(rotation=45)
plt.show()

# ANOVA para días de semana
f_stat, p_val = stats.f_oneway(*[df_semana[df_semana['Dia_Semana_Num'] == dia]['Cantidad_Vendida'] 
                                for dia in range(1, 6)])
print(f"\nANOVA para días de semana (Lunes a Viernes): F = {f_stat:.2f}, p-valor = {p_val:.4f}")

# Calcular estadísticas adicionales
eta_squared = (f_stat * (len(df_semana.groupby('Dia_Semana_Num')) - 1)) / (f_stat * (len(df_semana.groupby('Dia_Semana_Num')) - 1) + (len(df_semana) - len(df_semana.groupby('Dia_Semana_Num'))))

print("\nEstadísticas adicionales:")
print(f"Eta cuadrado: {eta_squared:.4f} (Tamaño del efecto)")

# Realizar prueba de Levene para homocedasticidad
levene_stat, levene_p = stats.levene(*[group['Cantidad_Vendida'].values for name, group in df_semana.groupby('Dia_Semana_Num')])
print(f"Test de Levene: estadístico = {levene_stat:.2f}, p-valor = {levene_p:.4f}")

print("\nInterpretación:")
print("- El p-valor extremadamente bajo (< 0.05) indica diferencias significativas en las ventas entre días")
print("- El valor F alto sugiere que la variación entre días es mucho mayor que dentro de cada día")
print(f"- El eta cuadrado (η²) de {eta_squared:.4f} indica que el {eta_squared*100:.1f}% de la varianza en ventas")
# Eta cuadrado (η²) es un indicador estadístico que mide el tamaño del efecto o la fuerza de la relación
# entre variables en un análisis de varianza (ANOVA). Técnicamente, representa la proporción de la
# variabilidad total en la variable dependiente (ventas) que puede ser explicada por la variable
# independiente (en este caso, el día de la semana). Los valores van de 0 a 1, donde:
# - η² cercano a 0 indica poco efecto
# - η² cercano a 1 indica un efecto muy fuerte
# Este coeficiente se suele representar con la letra griega η (eta)
print("  se explica por el día de la semana")


# Análisis de Variables Categóricas - Interpretación de Resultados

## Significancia Estadística

### Feriados
- **Prueba t**: t = 13.40, p-valor = 0.0000
- **Interpretación**: Existe una diferencia estadísticamente significativa en las ventas entre días feriados y no feriados. El p-valor extremadamente bajo (< 0.05) indica que esta diferencia no es producto del azar.

### Promociones
- **Prueba t**: t = 32.25, p-valor = 0.0000
- **Interpretación**: Las promociones de croissants tienen un impacto altamente significativo en las ventas. El valor t más alto (32.25) sugiere que el efecto de las promociones es aún más pronunciado que el de los feriados.

### Días de la Semana
- **ANOVA**: F = 111.56, p-valor = 0.0000
- **Interpretación**: Existen diferencias significativas en las ventas entre los diferentes días de la semana. El alto valor F (111.56) indica que la variabilidad entre días es mucho mayor que la variabilidad dentro de cada día.

### Meses
- **ANOVA**: F = 38.24, p-valor = 0.0000
- **Interpretación**: Se confirma la existencia de patrones estacionales en las ventas a lo largo del año. Aunque el efecto es significativo (p-valor < 0.05), el valor F más bajo en comparación con los días de la semana (38.24 vs 111.56) sugiere que la variación entre meses es menos pronunciada que la variación entre días de la semana.

## Conclusiones Principales

1. **Efecto de Promociones**: Las promociones de croissants tienen el impacto más fuerte en las ventas, con el valor t más alto (32.25).

2. **Patrón Semanal**: La variación entre días de la semana es la más pronunciada (F = 111.56), lo que sugiere que el día de la semana es un factor determinante en las ventas.

3. **Efecto de Feriados**: Los días feriados tienen un impacto significativo pero menos pronunciado que las promociones (t = 13.40).

4. **Estacionalidad**: Existe un patrón estacional claro, pero su impacto es menos pronunciado que los efectos diarios y semanales.

## Recomendaciones

1. **Optimización de Promociones**: Dado el fuerte impacto de las promociones, se recomienda analizar en detalle qué tipos de promociones son más efectivas y en qué momentos.

2. **Planificación por Día**: La fuerte variación semanal sugiere la necesidad de ajustar los niveles de inventario y personal según el día de la semana.

3. **Preparación para Feriados**: Aunque el efecto es menor que las promociones, los feriados siguen siendo importantes y requieren planificación especial.

4. **Gestión Estacional**: La estacionalidad mensual, aunque menos pronunciada, debe considerarse en la planificación a largo plazo y en la gestión de inventario.

# <div style="text-align: center;"><img src="./aux/kdd_5.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>
# <div style="text-align: center;"><img src="./aux/kdd_3a.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


## Modelado (Regresión Lineal) y Evaluación


# Tendencias y estructuras antes del modelado


Contra elementos temporales intra serie


### Modelos lineales

Un modelo lineal es una herramienta estadística que busca explicar una variable dependiente (Y) como una combinación lineal de variables independientes (X). Formalmente se representa como:

Y = β₀ + β₁X₁ + β₂X₂ + ... + βₖXₖ + ε

Donde:
- Y es la variable dependiente o respuesta
- X₁, X₂, ..., Xₖ son las variables independientes o predictoras
- β₀ es el intercepto o término constante
- β₁, β₂, ..., βₖ son los coeficientes que miden el efecto de cada variable X
- ε es el término de error aleatorio

El caso más simple es el modelo con un solo regresor:

Y = β₀ + β₁X + ε

Este modelo asume una relación lineal entre X e Y, donde β₀ representa el valor esperado de Y cuando X=0, y β₁ representa el cambio esperado en Y por cada unidad de cambio en X.

Los coeficientes β se estiman típicamente por el método de mínimos cuadrados ordinarios (OLS), que minimiza la suma de los errores al cuadrado.


In [ ]:
# 1. Regresión lineal simple para temperatura
X_temp = sm.add_constant(df['Temperatura_Max_Prevista'].astype(float))
y = df['Cantidad_Vendida'].astype(float)
modelo_temp = sm.OLS(y, X_temp).fit()
print("=== REGRESIÓN SIMPLE TEMPERATURA ===")
print(modelo_temp.summary())


In [ ]:

# Gráfico de regresión temperatura
plt.figure(figsize=(10, 6))
sns.regplot(x='Temperatura_Max_Prevista', y='Cantidad_Vendida', data=df)
plt.title('Regresión: Ventas vs Temperatura')
plt.show()


In [ ]:
df_reg = pd.DataFrame({
    'Temperatura': df['Temperatura_Max_Prevista'].astype(float),
    'Precio_Nuestro': df['Precio_Nuestro'].astype(float),
    'Precio_Competencia': df['Precio_Competencia'].astype(float),
    'Promocion': df['Promocion_Croissant'].astype(int),
    'Feriado': df['Es_Feriado'].astype(int),
    'Fin_Semana': df['Es_Fin_Semana'].astype(int),
    'Mes': df['Mes'].astype(int),
    'Dia_Semana': df['Dia_Semana_Num'].astype(int),
    'Cantidad_Vendida': df['Cantidad_Vendida'].astype(float)  # En términos absolutos, pero podríamos normalizar, logaritmizar, etc. 
})


# 5. Regresión polinomial para temperatura
df_reg['Temp_Cuad'] = df_reg['Temperatura'] ** 2
X_poly = df_reg[['Temperatura', 'Temp_Cuad']]
X_poly = sm.add_constant(X_poly)
modelo_poly = sm.OLS(df_reg['Cantidad_Vendida'], X_poly).fit()
print("\n=== REGRESIÓN POLINOMIAL TEMPERATURA ===")
print(modelo_poly.summary())

# Gráfico de regresión polinomial
plt.figure(figsize=(10, 6))
sns.regplot(x='Temperatura', y='Cantidad_Vendida', data=df_reg, 
            order=2, scatter_kws={'alpha':0.3})
plt.title('Regresión Polinomial: Ventas vs Temperatura')
plt.show()


### Modelos lineales con múltiples regresores

Extendiendo el modelo lineal simple a múltiples variables independientes, cada regresor adicional aporta información para explicar la variabilidad de Y:

Y = β₀ + β₁X₁ + β₂X₂ + ... + βₖXₖ + ε

Al incluir múltiples regresores:

- Se asume que la variable a explicar depende de múltiples elementos y no sólo de una variable
- Cada coeficiente βᵢ captura el efecto marginal de su variable Xᵢ sobre Y, manteniendo las demás constantes
- Variables relevantes adicionales ayudan a reducir el error y mejorar el ajuste
- Es posible modelar efectos directos e interacciones entre variables
- Permite controlar por factores confusores y aislar efectos individuales





In [ ]:
# 2. Regresión múltiple con variables temporales y categóricas
# Crear DataFrame para regresión con tipos explícitos
df_reg = pd.DataFrame({
    'Temperatura': df['Temperatura_Max_Prevista'].astype(float),
    'Precio_Nuestro': df['Precio_Nuestro'].astype(float),
    'Precio_Competencia': df['Precio_Competencia'].astype(float),
    'Promocion': df['Promocion_Croissant'].astype(int),
    'Feriado': df['Es_Feriado'].astype(int),
    'Fin_Semana': df['Es_Fin_Semana'].astype(int),
    'Mes': df['Mes'].astype(int),
    'Dia_Semana': df['Dia_Semana_Num'].astype(int),
    'Cantidad_Vendida': df['Cantidad_Vendida'].astype(float)  # En términos absolutos, pero podríamos normalizar, logaritmizar, etc. 
})

X_multi = df_reg[['Temperatura', 'Precio_Nuestro', 'Precio_Competencia', 
                'Promocion', 'Feriado', 'Fin_Semana', 'Mes']]
X_multi = sm.add_constant(X_multi)
modelo_multi = sm.OLS(df_reg['Cantidad_Vendida'], X_multi).fit()
print("\n=== REGRESIÓN MÚLTIPLE ===")
print(modelo_multi.summary())

In [ ]:

# 3. Regresión con variables dummy para días de la semana
dias_dummy = pd.get_dummies(df_reg['Dia_Semana'], prefix='Dia', drop_first=True).astype(int)
X_dias = pd.concat([X_multi, dias_dummy], axis=1)
modelo_dias = sm.OLS(df_reg['Cantidad_Vendida'], X_dias).fit()
print("\n=== REGRESIÓN CON DÍAS DE LA SEMANA ===")
print(modelo_dias.summary())


In [ ]:

# 4. Regresión con interacciones
df_reg['Temp_Promo'] = df_reg['Temperatura'] * df_reg['Promocion']
df_reg['Precio_Diff'] = df_reg['Precio_Nuestro'] - df_reg['Precio_Competencia']
X_inter = df_reg[['Temperatura', 'Precio_Nuestro', 'Precio_Competencia',
                'Promocion', 'Feriado', 'Fin_Semana', 'Mes',
                'Temp_Promo', 'Precio_Diff']]
X_inter = sm.add_constant(X_inter)
modelo_inter = sm.OLS(df_reg['Cantidad_Vendida'], X_inter).fit()
print("\n=== REGRESIÓN CON INTERACCIONES ===")
print(modelo_inter.summary())


# Regresiones con Series de Tiempo

Las regresiones con series de tiempo usando OLS (Ordinary Least Squares) son un buen punto de partida para analizar relaciones temporales, aunque presentan algunas limitaciones:

## Aproximación OLS Básica
- Asume independencia entre observaciones
- No considera autocorrelación ni heterocedasticidad
- Útil para primeras intuiciones y relaciones generales: simplemente introducimos como regresores variables de tiempo

## Alternativas más Robustas

### DOLS (Dynamic OLS)
- Incorpora rezagos y adelantos de variables
- Corrige problemas de endogeneidad
- Más apropiado para series no estacionarias cointegradas

### Fixed Effects (FE)
- Controla por heterogeneidad no observada constante en el tiempo
- Útil cuando hay efectos específicos por unidad/individuo
- Elimina sesgos por variables omitidas invariantes en el tiempo

### Random Effects (RE) 
- Asume efectos aleatorios no correlacionados con regresores
- Más eficiente que FE si se cumplen supuestos
- Permite estimar efectos de variables invariantes en el tiempo

Estas alternativas resuelven problemas de:
- Sesgo por endogeneidad y variables omitidas
- Ineficiencia por autocorrelación
- Inferencia incorrecta por heterocedasticidad

Para análisis más rigurosos, se recomienda considerar estas metodologías más avanzadas.


In [ ]:

# 6. Regresión con variables de tiempo
df_reg['Dia_Anio_Sin'] = np.sin(2 * np.pi * df['Dia_Anio'] / 365)
df_reg['Dia_Anio_Cos'] = np.cos(2 * np.pi * df['Dia_Anio'] / 365)
X_time = df_reg[['Temperatura', 'Precio_Nuestro', 'Precio_Competencia',
                'Promocion', 'Feriado', 'Fin_Semana',
                'Dia_Anio_Sin', 'Dia_Anio_Cos']]
X_time = sm.add_constant(X_time)
modelo_time = sm.OLS(df_reg['Cantidad_Vendida'], X_time).fit()
print("\n=== REGRESIÓN CON VARIABLES TEMPORALES ===")
print(modelo_time.summary())

# 7. Validación cruzada y métricas
X = df_reg[['Temperatura', 'Precio_Nuestro', 'Precio_Competencia',
            'Promocion', 'Feriado', 'Fin_Semana', 'Mes']]

# Estandarizar variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df_reg['Cantidad_Vendida'], 
                                                    test_size=0.2, random_state=42)

# Entrenar modelo
modelo_final = sm.OLS(y_train, sm.add_constant(X_train)).fit()
y_pred = modelo_final.predict(sm.add_constant(X_test))



In [ ]:
# Métricas
print("\n=== MÉTRICAS DEL MODELO ===")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")

# Gráfico de residuos
plt.figure(figsize=(10, 6))
sns.residplot(x=y_pred, y=y_test, lowess=True)
plt.title('Gráfico de Residuos')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuos')
plt.show()

---

# <div style="text-align: center;"><img src="./aux/kdd_5.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


---

    Sobre las regresiones... 

¿A qué fase del KDD pertenecen esos pasos?

Esos pasos pertenecen a la fase de Modelado Exploratorio.

 -		Cuando usamos regresión de manera exploratoria (antes de la división Train/Test formal), no estamos modelando para desplegar todavía.
 - 		Estamos estudiando la relación entre variables.
 - 		Estamos formándonos una idea de cómo es el fenómeno.

---


---
---


## Modelado y Evaluación

### KDD: Preprocesamiento - Limpieza Básica

**Discusión Limpieza:**
*   Nuestro dataset sintético está limpio. En un caso real, este paso es crucial.
*   Estrategias para nulos en series de tiempo: `ffill` (propagación hacia adelante), `bfill` (hacia atrás), interpolación (`.interpolate()`), imputación con media/mediana (simple, pero puede distorsionar patrones temporales).

### 2.3. Fase KDD: Transformación - Ingeniería de Features (1/2) (2/2: test-train data)

In [ ]:
describe_vars(df)

In [ ]:
# Copiamos el dataframe para no modificar el original durante el preproceso
df_processed = df.copy()

#####################################################


# --- Crear Dummies para variables categóricas ---


### otros ejemplos de variables categóricas:
## Categoría de producto (1:CROISSANT, 2:PAN, 3:TARTA, 4:PASTEL)
## Evaluación del servicio (Lickert "Calidad de Servicio" : muy mala, mala, buena, muy buena)
## Evaluación del servicio (Lickert "Calidad de Servicio" : 1, 2, 3, 4)
############################   no es irrelevante el orden de las categorías, ni el valor base que descartemos!!!! 
############################   es el caso de UNA VARIABLE CATEGÓRICA ORDINAL O ORDENADA





# Seleccionar columnas categóricas a codificar
categorical_features = ['Dia_Semana', 'Mes'] 

print(f"\nCodificando variables categóricas: {categorical_features}")

# One hot encoding - OHE
# Usamos pd.get_dummies para simplicidad. drop_first=True para evitar multicolinealidad perfecta. <<< 
df_processed = pd.get_dummies(df_processed, columns=categorical_features, drop_first=True, dtype=int)

print("\nNuevas columnas generadas:")
print(df_processed.columns)


In [ ]:
describe_vars(df_processed)
# Chequeo si las binarias originales son numéricas (0/1)

In [ ]:
# Ejemplo de protección de flujo:
# sirve para que evitar que si en otra situación esas variables vinieran mal definidas, por ejemplo:
	# como object (strings tipo “0”/“1”),
	# como boolean (True/False),
	

#vars target:
binary_cols = ['Es_Feriado', 'Promocion_Croissant']

#transformo en batch:
for col in binary_cols:
    if col in df_processed.columns and df_processed[col].dtype not in ['int64', 'float64', 'int32', 'float32']:
        print(f"Convirtiendo columna binaria '{col}' a tipo numérico (int)...")
        df_processed[col] = df_processed[col].astype(int)

print("\nTipos de datos finales después de transformaciones:")
df_processed.info()
describe_vars(df_processed)

> Nota: one-hot encoding sobre una variable categórica



En los casos "Dia_Semana", valores como "lunes", "martes", "miércoles", etc. se crean tantas columnas como categorías.


lunes	martes	miércoles
1	0	0
0	1	0
0	0	1
1	0	0

la suma de las tres columnas siempre da 1 → colinealidad perfecta.

Solución:
	•	drop_first=True en pd.get_dummies() elimina una columna (por ejemplo "lunes").
	•	Ahora si "martes" = 0 y "miércoles" = 0, ya sabés que era "lunes".
	•	Así se evita la colinealidad perfecta: no hay redundancia exacta.

Cuando se hace drop_first=True, se toma una categoría como referencia implícita (la primera, 0=lunes). Sin embargo ese drop no la hace desaparecer conceptualmente, sino que los efectos de las demás categorías se interpretan en relación a esa.


In [ ]:
describe_vars(df_processed)

**Discusión Ingeniería de Features:**
*   **One-Hot Encoding:** Convierte categorías (Lunes, Martes...) en columnas binarias (0/1) que los modelos lineales pueden usar.
*   **`drop_first=True`:** Elimina una categoría de cada grupo (ej. `Dia_Semana_Monday`) para evitar redundancia. El efecto de la categoría eliminada queda capturado en el intercepto del modelo.
*   **Otras Features (Para Futuro):**
    *   *Lags:* `df['Ventas_Ayer'] = df['Cantidad_Vendida'].shift(1)` - ¡Muy importante para series de tiempo!
    *   *Medias Móviles como Features:* `df['Ventas_MM_7dias'] = df['Cantidad_Vendida'].shift(1).rolling(7).mean()`
    *   *Features Cíclicas:* Usar `sin` y `cos` para `Mes` o `Dia_Anio` para capturar la ciclicidad de forma continua.
    *   *Interacciones:* `df['Precio_x_Promo'] = df['Precio_Nuestro'] * df['Promocion_Croissant']`
    *   *Tendencia Temporal:* Podríamos añadir una columna con un contador simple (`range(len(df))`) o usar `Anio`.

---

# Data Feature Design - Feature Engineering

El objetivo es esclarecer o brindar perspectivas alternativas sobre un fenómeno que se expresa en los datos.

- Todo dato puede revelar más información que la que su variable central inicialmente sugiere.
- Podemos segmentar las transformaciones en:
    - Transformaciones necesarias para interpretar el fenómeno.
    - Enfoques alternativos que ofrecen lecturas más ricas del fenómeno.
    - Transformaciones orientadas a fortalecer el modelo y su interpretación.

## Técnicas básicas:

- **Categorización**: convertir variables continuas en discretas para captar cambios clave (por ejemplo: muy alto, alto, medio, bajo) o crear agrupaciones específicas (grupo 1, grupo 2, etc.).
- **Normalización y Rescalamiento**: ajustar escalas para facilitar el procesamiento y la interpretación.
- **Deltas diferenciales y medias móviles**: capturar dinámicas y tendencias temporales o de variación entre observaciones.
- **Dummies (One-Hot Encoding)**: representación binaria (0-1, No-Si) de variables (categóricas o continuas) para una mejor integración en modelos estadísticos o de machine learning.

# ![Tabla de Transformaciones](./aux/tabla_transformaciones.png)


# Transformaciones Básicas

### 1. One-Hot Encoding (OHE) - Dummy Variables
**Qué hace**: convierte variables categóricas en variables binarias (0/1).  
**Cuándo usarlo**: siempre que quieras incorporar variables categóricas en modelos que requieren entradas numéricas (regresiones, árboles, redes neuronales).

### 2. Normalización [0, 1]
**Qué hace**: escala los valores de una variable para que estén entre 0 y 1.  
**Cuándo usarlo**: útil cuando las variables tienen escalas diferentes y usás modelos sensibles a la magnitud (ej: redes neuronales, KNN).

### 3. Estandarización (Media 0, Varianza 1)
**Qué hace**: centra la variable en media 0 y la escala para tener desviación estándar 1.  
**Cuándo usarlo**: fundamental cuando el modelo asume distribución normal o cuando se usan penalizaciones tipo Lasso/Ridge.

### 4. Categorización de Variables Continuas
**Qué hace**: convierte variables numéricas en grupos discretos o rangos (bins).  
**Cuándo usarlo**: para analizar no linealidades, facilitar interpretaciones o preparar variables para visualización.

### 5. Transformación Logarítmica
**Qué hace**: aplica logaritmo a los valores, reduciendo el efecto de valores extremos.  
**Cuándo usarlo**: útil cuando hay alta asimetría positiva o presencia de outliers grandes.

In [ ]:
# Transformaciones Básicas usando Categoria

# Copiar el dataframe de trabajo antes de las transformaciones para poder revertir y comparar
df_transf = df_processed.copy()


# --- 1. One-Hot Encoding sobre Categoria ---
print("\nCategorías únicas antes del One-Hot Encoding:")
print(df_transf['Categoria'].unique())

df_transf = pd.get_dummies(df_transf, columns=['Categoria'], drop_first=True, dtype=int)

print("\nColumnas nuevas después del One-Hot Encoding sobre Categoria:")
print([col for col in df_transf.columns if col.startswith('Categoria_')])

# --- 2. Normalización de Cantidad_Vendida ---
print("\nCantidad_Vendida - Antes de Normalizar:")
print(df_transf['Cantidad_Vendida'].describe())

df_transf['Cantidad_Vendida_normalizada'] = (df_transf['Cantidad_Vendida'] - df_transf['Cantidad_Vendida'].min()) / (df_transf['Cantidad_Vendida'].max() - df_transf['Cantidad_Vendida'].min())

print("\nCantidad_Vendida - Después de Normalizar:")
print(df_transf['Cantidad_Vendida_normalizada'].describe())

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
df_transf['Cantidad_Vendida'].hist(bins=30)
plt.title('Cantidad_Vendida Original')

plt.subplot(1,2,2)
df_transf['Cantidad_Vendida_normalizada'].hist(bins=30)
plt.title('Cantidad_Vendida Normalizada')
plt.tight_layout()
plt.show()

# --- 3. Estandarización de Cantidad_Vendida ---
df_transf['Cantidad_Vendida_estandarizada'] = (df_transf['Cantidad_Vendida'] - df_transf['Cantidad_Vendida'].mean()) / df_transf['Cantidad_Vendida'].std()

print("\nCantidad_Vendida - Después de Estandarizar:")
print(df_transf['Cantidad_Vendida_estandarizada'].describe())

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
df_transf['Cantidad_Vendida'].hist(bins=30)
plt.title('Cantidad_Vendida Original')

plt.subplot(1,2,2)
df_transf['Cantidad_Vendida_estandarizada'].hist(bins=30)
plt.title('Cantidad_Vendida Estandarizada')
plt.tight_layout()
plt.show()

# --- 4. Categorización de Cantidad_Vendida ---
df_transf['Cantidad_Vendida_categoria'] = pd.cut(df_transf['Cantidad_Vendida'],
                                                  bins=[-np.inf, 100, 500, 1000, np.inf],
                                                  labels=['Muy Baja', 'Baja', 'Media', 'Alta'])

print("\nDistribución de Categorías en Cantidad_Vendida:")
print(df_transf['Cantidad_Vendida_categoria'].value_counts())

# --- 5. Logaritmo de Cantidad_Vendida ---
df_transf['Cantidad_Vendida_log'] = np.log1p(df_transf['Cantidad_Vendida'])

print("\nCantidad_Vendida - Después de Log-Transformar:")
print(df_transf['Cantidad_Vendida_log'].describe())

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
df_transf['Cantidad_Vendida'].hist(bins=30)
plt.title('Cantidad_Vendida Original')

plt.subplot(1,2,2)
df_transf['Cantidad_Vendida_log'].hist(bins=30)
plt.title('Cantidad_Vendida Log-Transformada')
plt.tight_layout()
plt.show()

# <div style="text-align: center;"><img src="./aux/kdd_5.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>
# <div style="text-align: center;"><img src="./aux/kdd_4.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# FASE TRANSFORMACIÓN (2/2)

## Preparación e implementación de un Modelo

---

### 3.1. Fase KDD: Modelado y Evaluación (Inicio)


#### Fase KDD: Preparación para el Modelado

- División temporal Train/Test
- Entrenamiento del modelo
- Predicción
- Evaluación de desempeño

#### 3.1.1. División Train/Test 

### División Temporal del Dataset: ¿Qué hacemos, por qué y cómo?

#### ¿Qué hacemos?

Dividimos el dataset en dos subconjuntos: 
- Un **conjunto de entrenamiento (Train)**: contiene los datos más antiguos.
- Un **conjunto de prueba (Test)**: contiene los datos más recientes.

Utilizamos la **última porción de tiempo (últimos 365 días)** como test, mientras que el resto queda como entrenamiento.

---

#### ¿Por qué lo hacemos?

La razón principal es simular un escenario realista de **predicción futura**:
- En la práctica, cuando hacemos pronósticos o predicciones, siempre intentamos predecir **hacia adelante en el tiempo** usando sólo información **del pasado**.
- Si mezcláramos aleatoriamente los datos, estaríamos "viendo" el futuro antes de tiempo, y eso generaría **falsas buenas predicciones**.
- Dividir temporalmente respeta la **secuencia cronológica** y evita **fugas de información** (data leakage).

---

#### ¿Cómo lo hacemos?

1. **Seleccionamos la columna objetivo** que queremos predecir, en este caso `'Cantidad_Vendida'`.

2. **Definimos la fecha de corte**:
   - Tomamos el último registro de fecha del dataset.
   - Retrocedemos 365 días para establecer el punto donde se separan los datos de entrenamiento y prueba.

3. **Creamos dos conjuntos**:
   - `X_train`, `y_train`: datos anteriores al corte → sirven para entrenar los modelos.
   - `X_test`, `y_test`: datos posteriores o iguales al corte → sirven para evaluar cómo se desempeña el modelo con datos que "nunca vio".

4. **Visualizamos** la división temporal:
   - Mostramos las series de entrenamiento y prueba en un mismo gráfico.
   - Marcamos claramente la línea de división para entender visualmente qué parte del tiempo corresponde a cada conjunto.

---

#### Resumen

Esta división **simula condiciones reales de predicción**:
- Entrenamos el modelo **sólo con el pasado**.
- Evaluamos el modelo **en el futuro**, como ocurre en aplicaciones reales.
- Respetamos la lógica temporal de los datos para obtener resultados más **realistas**, **fiables** y **útiles**.

### Variables de Predicción:

In [134]:
# --- Definir Predictores (X) y Objetivo (y) ---
    # como hicimos en el EDA para los modelos de regresión 

# Excluimos la columna objetivo
target_column = 'Cantidad_Vendida'

try:
    X = df_processed.drop(columns=[target_column])
    y = df_processed[target_column]
except KeyError:
    print(f"Error: La columna objetivo '{target_column}' no se encuentra en df_processed.")
    raise



	•	Crear una nueva variable llamada X. (X=...) --> X es una copia del df_processed pero sin la columna 'Cantidad_Vendida'.
    - 	Y = ... --> es sólo la columna 'Cantidad_Vendida'.

    Por qué?

    Los modelos necesitan separar claramente input (X) de output (y).


### Split del dataset para training

In [ ]:

# --- División Temporal --- 
# Usaremos el último año (365 días) como conjunto de test
test_size_days = 365 
split_date = X.index.max() - pd.DateOffset(days=test_size_days - 1)

X_train = X[X.index < split_date]
X_test = X[X.index >= split_date]
y_train = y[y.index < split_date]
y_test = y[y.index >= split_date]

print(f"--- División Temporal Realizada ---")
print(f"Tamaño del set de entrenamiento (X_train): {X_train.shape}")
print(f"Tamaño del set de entrenamiento (y_train): {y_train.shape}")
print(f"Tamaño del set de test (X_test):        {X_test.shape}")
print(f"Tamaño del set de test (y_test):        {y_test.shape}")

print(f"\nFecha de inicio de Train: {X_train.index.min().strftime('%Y-%m-%d')}, Fecha de fin de Train: {X_train.index.max().strftime('%Y-%m-%d')}")
print(f"Fecha de inicio de Test:  {X_test.index.min().strftime('%Y-%m-%d')}, Fecha de fin de Test:  {X_test.index.max().strftime('%Y-%m-%d')}")

# Visualizar la división
plt.figure(figsize=(16, 4))
y_train.plot(label='Train Set')
y_test.plot(label='Test Set')
plt.axvline(split_date, color='black', linestyle='--', label=f'División ({split_date.strftime("%Y-%m-%d")})')
plt.title('División Temporal Train/Test de la Variable Objetivo')
plt.ylabel(target_column)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

### Para este ejemplo, tiene sentido: 

- Último año implica que tengo un corte claro para predecir la tendencia 
- Implica que mi predicción se va a basar y a entrenar sobre una serie concisa y que "estira" (estima) el período siguiente
- Los datos sobre la estimación tienen una validación explícita. 
- ...cuidado, esto no siempre es así. 

---


##### ¿Cuál es el problema de usar siempre el “último año”?
	•	Puede estar afectado por eventos atípicos: pandemia, inflación alta, cambios normativos, catástrofes, etc.
	•	Puede no ser representativo de la dinámica histórica general.
	•	Puede dificultar la generalización si queremos construir un modelo robusto para distintos momentos históricos.


# ![Tabla Training](./aux/tabla_training.png)


In [ ]:
# Guardamos la versión final del dataframe en parquet para usarlo en el despliegue

# !pip install pyarrow

# Then save and load parquet

# Save df_processed as parquet
df_processed.to_parquet('df_processed.parquet')

# Load df_processed from parquet
df_processed = pd.read_parquet('df_processed.parquet')

# Describe variables
describe_vars(df_processed)


# <div style="text-align: center;"><img src="./aux/kdd_5.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# Fase 4: Modelización y Predicción - Minería de datos


## Despliegue predictivo

En esta etapa central del proceso KDD, nos centraremos en el despliegue del modelo predictivo para la demanda de croissants. El despliegue implica la implementación del modelo en un entorno de producción donde pueda ser utilizado para realizar predicciones en tiempo real y ayudar en la toma de decisiones del negocio.

### Objetivos del Despliegue

1. **Preparación del Modelo**:
   - Serializar el modelo entrenado para su uso posterior
   - Establecer un pipeline de predicción claro y reproducible
   - Documentar los requisitos y dependencias

2. **Implementación**:
   - Crear funciones de predicción fáciles de usar
   - Establecer un flujo de trabajo para nuevas predicciones
   - Asegurar la consistencia en el procesamiento de datos

3. **Monitoreo y Mantenimiento**:
   - Definir métricas de monitoreo del rendimiento
   - Establecer procedimientos de actualización del modelo
   - Implementar validaciones de calidad de datos

### Consideraciones Importantes

- **Reproducibilidad**: Asegurar que el proceso sea reproducible en diferentes entornos
- **Escalabilidad**: Preparar el sistema para manejar diferentes volúmenes de predicciones
- **Mantenibilidad**: Facilitar la actualización y mejora continua del modelo
- **Documentación**: Proporcionar documentación clara para usuarios y desarrolladores

## Modelo: Decision Trees

# ![Tabla de modelos de decision trees](./aux/tabla_modelos_dt1.png)
# 


In [ ]:


# Variables predictoras y target
features = [
    'Temperatura_Max_Prevista',
    'Promocion_Croissant',
    'Precio_Nuestro',
    'Precio_Competencia',
    'Es_Feriado',
    'Es_Fin_Semana',
    'Dia_Semana_jueves',
    'Dia_Semana_lunes',
    'Dia_Semana_martes',
    'Dia_Semana_miércoles',
    'Dia_Semana_sábado',
    'Dia_Semana_viernes',
    'Mes_2', 'Mes_3', 'Mes_4', 'Mes_5', 'Mes_6', 'Mes_7',
    'Mes_8', 'Mes_9', 'Mes_10', 'Mes_11', 'Mes_12'
]
target = 'Cantidad_Vendida'

X = df_processed[features]
y = df_processed[target]

# División en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del árbol de decisión
tree_model = DecisionTreeRegressor(max_depth=4, random_state=42,  max_leaf_nodes=12)
tree_model.fit(X_train, y_train)



### ¿Qué hace el modelo DecisionTreeRegressor?

Un DecisionTreeRegressor es un modelo de aprendizaje supervisado (Supervised Machine Learning) que predice valores numéricos continuos. En este caso, el valor a predecir es Cantidad_Vendida de croissants. Funciona dividiendo el espacio de datos en particiones jerárquicas basadas en reglas del tipo:
	•	¿Precio_Nuestro < 80?
	•	¿Es_Fin_Semana == 1?
	•	¿Temperatura_Max_Prevista > 27?

Cada nodo del árbol representa una condición que separa el dataset según un umbral de una variable. Los nodos terminales (hojas) contienen el valor promedio de la variable objetivo (Cantidad_Vendida) en esa partición.

⸻

### ¿Para qué sirve?
	Explicabilidad: al ser gráfico e interpretable, es ideal para entender la lógica detrás de las predicciones.
	Entrenamiento base: sirve como modelo simple antes de usar RandomForest, que es una técnica apoyada sobre un conjunto de árboles como este.

⸻

### ¿Qué muestra?

Al visualizar el árbol:
	•	Cada nodo muestra una regla: p. ej. Promoción <= 0.5
	•	Los nodos hijos indican la partición resultante si la condición se cumple o no.
	•	Los valores en las hojas indican la predicción promedio de Cantidad_Vendida para ese subconjunto.

También se incluye el número de observaciones en cada nodo y la desviación estándar.

⸻

### ¿Cómo se interpreta?
	•	Un árbol permite ver las reglas más importantes que dividen los datos.
	•	El orden de las condiciones da una idea de qué variables son más relevantes en las decisiones del modelo.
	•	Si una hoja tiene muchas observaciones y una predicción alejada del promedio global, implica señales de overfitting (relacionado usualmente con la profundidad (parámetro max_depth=4, o leafs mínimas, o relación sample-vars-tree).



In [ ]:

# indicadores básicos del modelo
y_pred = tree_model.predict(X_test)
print("\n========\n","MSE del modelo (dv std predicto versus validation):", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred), "\n========\n")

# Visualización en texto del arbol 
tree_text = export_text(tree_model, feature_names=features, decimals=1)
print(tree_text)



# ATENCIÓN:

Los árboles de decisión, como `DecisionTreeRegressor` de scikit-learn, siempre dividen el conjunto de datos mediante **reglas binarias**, es decir, cada nodo se divide en dos ramas según una condición.

Veamos los distintos tipos de variables al construir los splits:

---

**1. Variables binarias (0 y 1):**

El árbol no reconoce que la variable es binaria. La trata como continua y evalúa posibles umbrales. Naturalmente, el único corte útil es:

$Variable \leq 0.5$ (equivale a: Variable == 0)

Aunque el resultado es correcto, la interpretación humana es más clara si se expresa como igualdad.

*- Interpretar las condiciones sobre variables binarias (como `<= 0.5`) como igualdades lógicas*

---

**2. Variables continuas (por ejemplo, temperatura entre 3 y 35):**

El árbol evalúa todos los puntos posibles entre valores consecutivos. Por cada posible umbral $t$, calcula:

$MSE_{split}(t) = \frac{n_L}{n} \cdot MSE_L + \frac{n_R}{n} \cdot MSE_R$

y selecciona el corte que minimiza ese valor. El resultado es una condición binaria del tipo:

$Temperatura \leq 18.5$


Algo un poco más detallado:

### Cómo se determinan los umbrales de corte para variables continuas en árboles de decisión

Cuando una variable es continua, el árbol **no evalúa todos los valores reales posibles** como umbrales (eso sería infinito). En cambio, sigue una estrategia concreta:

1. Ordena las observaciones según los valores de la variable continua $x_j$.
2. Considera como umbrales candidatos todos los **puntos medios entre pares de valores consecutivos distintos**.

Si los valores únicos ordenados de $x_j$ son $x_1, x_2, \dots, x_n$, los umbrales que el árbol evalúa son:

$t_k = \frac{x_k + x_{k+1}}{2}, \quad \text{para } k = 1, 2, \dots, n - 1$


#### Ejemplo:

Si $x = [3, 7, 12, 20]$, entonces los cortes candidatos son:

- $t_1 = \frac{3 + 7}{2} = 5$
- $t_2 = \frac{7 + 12}{2} = 9.5$
- $t_3 = \frac{12 + 20}{2} = 16$

##### Evaluación del mejor corte

Para cada candidato $t$, el árbol divide el conjunto de datos en dos subconjuntos:

- $S_L = \{ x_i \leq t \}$
- $S_R = \{ x_i > t \}$

y calcula el siguiente valor de error:

$MSE_{split}(t) = \frac{n_L}{n} \cdot MSE_L + \frac{n_R}{n} \cdot MSE_R$

donde:

- $n$ es el número total de observaciones
- $n_L$ y $n_R$ son los tamaños de los subconjuntos izquierdo y derecho
- $MSE_L$ y $MSE_R$ son los errores cuadráticos medios dentro de cada subconjunto

El árbol elige el umbral $t$ que **minimiza este valor** (por ej: `Temperatura_Max_Prevista <= 18.9`).

    Entonces:
    - Las divisiones del árbol **siempre son binarias**, incluso en variables continuas.
    - Los cortes se prueban **sólo en puntos medios entre valores únicos consecutivos**.
    - Esto garantiza que el espacio de búsqueda sea finito, exhaustivo y computacionalmente viable.


---

**3. Variables categóricas con más de dos valores (por ejemplo: A, B, C, D, E):**

Scikit-learn **no admite directamente variables categóricas**. 

Por tanto, deben transformarse:

- **OHE - Codificación one-hot (es decir, dummies, o dummy variables, variables 0,1):**

    crea una columna por categoría (`cat_A`, `cat_B`, etc.). El árbol realiza splits binarios sobre cada dummy:

    $cat\_B \leq 0.5$ (equivale a: categoría distinta de B)

- **Codificación ordinal (label encoding):** no se recomienda salvo que el orden tenga significado (poco-mucho, bajo-alto, bueno-malo y ese tipo de gradientes), ya que el árbol evaluará cortes continuos... esto hará que, por ejemplo, encontremos cortes como:

$Categoría \leq 2.5$

lo cual puede inducir a errores de interpretación. Lo más recomendable a no ser que haya una hipótesis particular de trabajo, es la transformación OHE. 
*- No usar variables categóricas sin transformar*
*- Aplicar codificación one-hot a todas las variables nominales no ordinales*




In [ ]:
# indicadores básicos del modelo
y_pred = tree_model.predict(X_test)
print("\n========\n","MSE del modelo (dv std predicto versus validation):", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred), "\n========\n")

# Visualización en texto del arbol 
tree_text = export_text(tree_model, feature_names=features, decimals=2)
print(tree_text)

In [ ]:
# Visualización del árbol (versión rústica, puede haber problemas de superposición)
plt.figure(figsize=(24, 14))
plot_tree(tree_model, feature_names=features, filled=True, rounded=True, fontsize=13)
plt.title("Árbol de decisión para predicción de cantidad vendida de croissants")
plt.show()

In [ ]:
# Visualización del árbol (versión elaborada, elimina superposiciones y genera más control)

output_dir = "model_outputs"
os.makedirs(output_dir, exist_ok=True)

# Rutas de salida
dot_path = os.path.join(output_dir, "arbol_croissants.dot")
png_path = os.path.join(output_dir, "arbol_croissants.png")

# Exportar el árbol a .dot
export_graphviz(
    tree_model,
    out_file=dot_path,
    feature_names=features,
    filled=True,
    rounded=True,
    special_characters=True
)

# Convertir .dot a .png
subprocess.run(["dot", "-Tpng", dot_path, "-o", png_path], check=True)

print(f"Árbol exportado en:\n- DOT: {dot_path}\n- PNG: {png_path}")
# Mostrar imagen generada
display(Image(filename="model_outputs/arbol_croissants.png"))

In [ ]:
# indicadores básicos del modelo
y_pred = tree_model.predict(X_test)
print("\n========\n","MSE del modelo (dv std predicto versus validation):", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred), "\n========\n")

# Visualización en texto del arbol 
tree_text = export_text(tree_model, feature_names=features, decimals=2)
print(tree_text)

# <div style="text-align: center;"><img src="./aux/kdd_6.png" width="400" height="300" style="filter: drop-shadow(5px 5px 5px rgba(0,0,0,0.3));"></div>


# Fase 5: Evaluación e interpretación


### Evaluación del MSE (primera aproximación, en el próximo encuentro veremos estrategias complementarias)

El Error Cuadrático Medio (MSE) nos devuelve un valor absoluto (ej. 234,9), pero ¿qué quiere decir? ¿Es bueno? ¿Es malo?

Por sí solo, el MSE es difícil de interpretar porque depende de la escala de la variable objetivo. Por eso, se utilizan métricas complementarias que permiten contextualizar ese error en términos relativos o porcentuales.

---

#### 1. **nRMSE** (Normalized Root Mean Squared Error)

$nRMSE = \frac{\sqrt{MSE}}{\sigma_y}$

- **Qué mide:** el RMSE mide cuánto se desvía (el error de) la predicción en relación al desvío estándar de la variable real.
- **Interpretación:** expresa el error como fracción de la variabilidad natural de los datos.  
  Por ejemplo, si $nRMSE = 0.2$, el error es el 20% de la dispersión típica de los valores reales.

¿Cuánto del total de la variación en y_test (dato real) no estoy pudiendo predecir correctamente?

Un $nRMSE < 1$ quiere decir que el error promedio de mis predicciones es menor que la variación total del fenómeno. ¿Por qué es esto posible? **Porque el modelo logra explicar parte de esa variación sistemáticamente usando las variables X.**


---

#### 2. **MASE** (Mean Absolute Scaled Error)

$MASE = \frac{MAE_{modelo}}{MAE_{naïve}}$

- **Qué mide:** compara el error absoluto medio del modelo con el que se obtendría si se predijera siempre la media de los valores reales.
- **Interpretación:** evalúa si el modelo, usando variables explicativas, logra reducir el error frente a una estrategia que no usa ninguna información.
  - $MASE < 1$: el modelo mejora respecto a predecir siempre la media.
  - $MASE = 1$: el modelo tiene el mismo error que predecir la media.
  - $MASE > 1$: el modelo es peor que simplemente predecir el promedio.
- **Importante:** no se compara la media de las predicciones, sino el error promedio respecto de los valores reales. No se espera que el modelo "reproduzca" el fenómeno, sino que lo prediga mejor que una estrategia vacía. Esa predicción implica las mejoras usadas por las features (variables X, o regresores)

---

#### 3. **SMAPE** (Symmetric Mean Absolute Percentage Error)

$SMAPE = \frac{100\%}{n} \sum_{i=1}^{n} \frac{|y_i - \hat{y}_i|}{(|y_i| + |\hat{y}_i|)/2}$

- **Qué mide:** el error porcentual promedio entre la predicción y el valor real.
- **Ventajas:** está acotado entre 0 y 100, y es más robusto cuando $y \to 0$ que el MAPE.
- **Interpretación:** un $SMAPE = 15\%$ indica que, en promedio, la diferencia entre predicción y valor real es del 15% del valor medio de ambos.
- 	Si el modelo captura toda la varianza de y (o sea, R² = 1), entonces el error es cero. El error sólo puede ser == 0 si la predicción es perfecta, es decir, consideramos todos los elementos (visibles, invisibles, directos e indirectos) que afectan cómo se mueve "y_test".


---

In [ ]:
# Indicadores adicionales de calidad del modelo

# nRMSE: RMSE normalizado por el desvío estándar de y_test
# Normalized Root Mean Squared Error - nRMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
std_y = np.std(y_test)
nrmse = rmse / std_y
print("\nIndicadores de calidad del modelo:")
print("---------------------------------")
print("nRMSE (RMSE / STD):", round(nrmse, 4))
print("Interpretación: El error promedio representa un {:.1f}% de la variabilidad natural de las ventas".format(nrmse*100))
print("Un nRMSE < 1 indica que el modelo logra explicar parte de la variación usando las variables predictoras")


# MASE: compara el MAE del modelo contra un pronóstico ingenuo (media)
# Mean Absolute Scaled Error - MASE
mae_modelo = mean_absolute_error(y_test, y_pred)
mae_naive = mean_absolute_error(y_test, np.full_like(y_test, np.mean(y_test)))
mase = mae_modelo / mae_naive
print("\nMASE (MAE modelo / MAE naive):", round(mase, 4))
if mase < 1:
    print("Interpretación: El modelo mejora en un {:.1f}% respecto a predecir la media de y_test".format((1-mase)*100))
    print("o: El uso de mis features (variables X) mejoran en un {:.1f}% la predicción respecto al error MAE de la media de y_test".format((1-mase)*100))

else:
    print("Interpretación: El modelo es peor que simplemente predecir la media porque es mayor a 1")

# SMAPE: error porcentual absoluto simétrico
# Symmetric Mean Absolute Percentage Error - SMAPE
smape = np.mean(
    np.abs(y_test - y_pred) / ((np.abs(y_test) + np.abs(y_pred)) / 2)
) * 100
print("\nSMAPE (%):", round(smape, 2))
print("Interpretación: En promedio, la diferencia entre predicción y valor real es del {:.1f}%".format(smape))
print("O: Frente a una predicción perfecta (0% de error) el error del modelo es del {:.1f}% (modelo tan bueno como tirar una moneda = error 100%)".format(smape))
print("Nota: si el error = 100%, el error promedio igualó al tamaño del fenómeno que se quería predecir. El modelo es inútil.")

In [ ]:
# indicadores básicos del modelo
y_pred = tree_model.predict(X_test)


print("\n========","\nMSE del modelo:", mean_squared_error(y_test, y_pred))
# nRMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
std_y = np.std(y_test)
nrmse = rmse / std_y
print("nRMSE:", round(nrmse, 4))
# MASE
mae_modelo = mean_absolute_error(y_test, y_pred)
mae_naive = mean_absolute_error(y_test, np.full_like(y_test, np.mean(y_test)))
mase = mae_modelo / mae_naive
print("MASE:", round(mase, 4))
# SMAPE
smape = np.mean(
    np.abs(y_test - y_pred) / ((np.abs(y_test) + np.abs(y_pred)) / 2)
) * 100
print("SMAPE (%):", round(smape, 2))
print("R²:", r2_score(y_test, y_pred))
print("========\n")


# Visualización en texto del arbol 
tree_text = export_text(tree_model, feature_names=features, decimals=2)
print(tree_text)

---

# Qué buscamos en la interpretación

### 1. Segmentos con alta demanda esperada  
**(Identificables visualmente en las hojas finales o vía SHAP/Path Extraction)**  
- Sirven para previsión, planificación de stock y logística  
    - `Promoción + Feriado + Temperatura alta → 240.00 unidades`

### 2. Segmentos con baja demanda esperada  
**(Visualización directa del árbol o filtrado por path predictivo con menor valor)**  
- Sirven para evitar sobreproducción o activar acciones comerciales  
    - `Sin promoción + No viernes + Temperatura baja → 67.08`

### 3. Sensibilidad a ciertas variables  
**(Evaluable mediante análisis de Gain o reducción local de MSE por split, o con SHAP local)**  
- Casos donde una sola variable cambia fuertemente la predicción  
    - `entre Es_Feriado = 0 y Es_Feriado = 1 con todo igual, pasás de 95.43 a 150.31`

### 4. Efectos acumulativos  
**(Extraíbles vía análisis de paths completos o combinaciones de condiciones en SHAP)**  
- ¿Qué pasa cuando se combinan variables favorables?  
    - Promoción + Fin de semana + Temperatura > 15.95 → 198.62  
    - + Feriado → 240.00

### 5. Umbrales de decisión prácticos  
**(Derivados directamente de los cortes en cada nodo del árbol: temperatura, precio, etc.)**  
- Temperatura ≤ o > 18  
- Precio ≤ 1.46  
- Son cortes concretos que te permiten anticipar cambios en el nivel de ventas

---

In [ ]:

# Métricas  para interpretar la estructura del árbol de decisión
# =============================================================================

# importancia de variables (gain), 
# reducción de error (impureza), 
# caminos de decisión por muestra (paths) y 
# valores SHAP.



# 1. Importancia de las variables (gain acumulado del árbol) >>> Métrica elemental e indispensable
# Esta métrica mide cuánto reduce el MSE cada variable a lo largo de todo el árbol.
importancia = pd.DataFrame({
    'Variable': features,
    'Importancia': tree_model.feature_importances_
}).sort_values(by='Importancia', ascending=False)

print("\nRESULTADOS DEL DECISION TREE")

print("\n--- Importancia de variables (Gain total por split) ---")
print(importancia)
# ¿Cuáles son las variables que más impactan en la predicción de ventas a nivel global del modelo?



# 2. Importancia por permutación (Permutation Importance)
# Evalúa cuánto empeora la performance del modelo si se desordena cada variable.
# Captura interacciones y dependencias más allá del árbol construido.
perm_importance = permutation_importance(tree_model, X_test, y_test, n_repeats=10, random_state=42)

importancia_perm = pd.DataFrame({
    'Variable': features,
    'Importancia': perm_importance.importances_mean
}).sort_values(by='Importancia', ascending=False)

print("\n--- Importancia por permutación (Permutation Importance) ---")
print(importancia_perm)
# ¿Qué variables son realmente importantes cuando se consideran las interacciones entre ellas?






# 3. Caminos de decisión (Decision Paths)
# Extrae las condiciones que se cumplen para cada muestra de test y el valor predicho
# Útil para identificar qué combinaciones activan ciertas predicciones
print("\n--- Caminos de decisión por observación ---")
tree_ = tree_model.tree_
feature_names = np.array(features)

def extraer_paths(model, X):
    paths = []
    for i in range(X.shape[0]):
        node_index = model.decision_path(X.iloc[[i]]).indices
        conds = []
        for node_id in node_index:
            if node_id == tree_.node_count - 1:
                continue
            feature = tree_.feature[node_id]
            threshold = tree_.threshold[node_id]
            if X.iloc[i, feature] <= threshold:
                conds.append(f"{feature_names[feature]} <= {threshold:.2f}")
            else:
                conds.append(f"{feature_names[feature]} > {threshold:.2f}")
        paths.append(" ∧ ".join(conds))
    return paths

# Generamos todos los paths
decision_paths = extraer_paths(tree_model, X_test)

# Muestras individuales (las primeras 5 en orden natural)
print("\n--- Caminos de decisión por observación (primeras 15 muestras) ---")
for i, path in enumerate(decision_paths[:15]):
    print(f"Muestra {i+1} (predicción: {round(y_pred[i], 2)}):")
    print(path)

print("\n\nSignifica que, dadas las condiciones observadas en esa muestra (Promoción = 0, Es fin de semana = 1, etc.), el árbol predice que se venderán aproximadamente 107.05 unidades.")

# Muestras ordenadas por valor predicho (descendente)
print("\n--- Caminos de decisión por predicción (ordenados) ---")
sorted_indices = np.argsort(-y_pred)  # mayor a menor

for rank, i in enumerate(sorted_indices[:5]):
    print(f"Muestra {i} (rank {rank+1}, predicción: {round(y_pred[i], 2)}):")
    print(decision_paths[i])
    





# 4. SHAP values para árbol
# Mide la contribución exacta de cada variable a cada predicción
explainer = shap.TreeExplainer(tree_model)
shap_values = explainer.shap_values(X_test)

## Graph Shap

# Filtro: eliminar variables con impacto nulo
shap_importancia = np.abs(shap_values).mean(axis=0)
variables_con_impacto = shap_importancia > 0
X_shap = X_test.loc[:, variables_con_impacto]
features_shap = X_shap.columns.tolist()

custom_cmap = LinearSegmentedColormap.from_list("gray_to_red", ["#d3d3d3", "#ff0000"])

# Visualización SHAP con plot ajustado
shap.summary_plot(
    shap_values[:, variables_con_impacto],
    X_shap,
    feature_names=features_shap,
    plot_size=(8, 5),
    cmap=custom_cmap
)
# Nota: la visualización SHAP abre una figura interactiva. Si corrés esto en un entorno no interactivo (como consola), podrías no verla.

---

	•	La mayoría de los datos de test no tienen promoción activa (Promocion_Croissant = 0), por eso hay más puntos azules.
	•	Pocas observaciones tienen valor alto (mayor a 1), de ahí los pocos puntos rojos.
    	•	Cuando hay promoción (puntos rojos), el impacto sobre la predicción es muy alto y positivo.
	•	Pero como hay menos casos con promoción, la frecuencia de esos puntos es menor.
	•	En cambio, la ausencia de promoción (azul) no baja tanto la predicción, sino que la mantiene neutral o levemente baja.
     frecuencia ≠ importancia, y eso se ve en cómo se dispersan los puntos.

---

# Recordemos: 

En cada nodo sucede esto:
$$
\text{Split óptimo} = \arg\min_{\text{split}} \left( \frac{n_{\text{izq}}}{n} \cdot \text{Var}_{\text{izq}}(y) + \frac{n_{\text{der}}}{n} \cdot \text{Var}_{\text{der}}(y) \right)
$$

---
### 1. Importancia de variables (Gain total por split)

| Variable | Importancia |
|----------|------------|
| Promocion_Croissant | 0.4228 |
| Es_Fin_Semana | 0.2836 |
| Temperatura_Max_Prevista | 0.1992 |
| Es_Feriado | 0.0700 |
| Dia_Semana_viernes | 0.0182 |
| Precio_Nuestro | 0.0062 |
| Resto de variables | 0.0000 |

Este indicador muestra cuánto contribuye cada variable a reducir el error (MSE) en los splits del árbol. Refleja la importancia estructural del modelo entrenado.

Interpretación:
- Promocion_Croissant es la variable dominante: representa el 42% de la ganancia total del modelo. Es el principal driver.
- Es_Fin_Semana y Temperatura_Max_Prevista también tienen peso significativo (28% y 20%).
- Es_Feriado tiene impacto, pero menor (7%).
- Las variables de día y mes, salvo viernes, no fueron usadas en ningún split, o tienen una contribución nula.

---

### 2. Permutation Importance

| Variable | Importancia |
|----------|------------|
| Promocion_Croissant | 0.9298 |
| Es_Fin_Semana | 0.3939 |
| Temperatura_Max_Prevista | 0.2790 |
| Es_Feriado | 0.1076 |
| Dia_Semana_viernes | 0.0179 |
| Precio_Nuestro | 0.0073 |
| Resto de variables | 0.0000 |

#### Qué es:

Es un método agnóstico del modelo que evalúa cuán dependiente es el modelo de cada variable para hacer predicciones correctas.

Para cada variable, se desordena aleatoriamente su columna en X_test. Esto rompe la relación entre esa variable y y, manteniendo el resto igual.

Se vuelve a predecir usando el modelo y se calcula cuánto empeora el error respecto al original. Este procedimiento se repite varias veces (n_repeats=10) para reducir ruido, y se promedia el impacto.

	•	Si el error aumenta mucho al desordenar una variable → esa variable es importante para el modelo.
	•	Si el error no cambia → el modelo no depende de esa variable.
    	Captura el impacto real sobre la predicción, no sobre la estructura.

 Es un test independiente de la estructura del árbol y más sensible a interacciones o efectos marginales no reflejados en splits explícitos.

#### Interpretación:
- Confirma el resultado anterior: Promoción es absolutamente determinante, con casi el 93% de la importancia relativa.
- Las demás mantienen su peso, aunque la magnitud relativa cambia un poco.
- Refuerza la conclusión de que muchas variables disponibles no aportan nada útil al modelo.

---

### 3. Caminos de decisión por observación (Decision Paths)

¿Qué conjunción de condiciones es necesaria para llegar a cierto resultado? Este análisis es estructural, no marginal ni de variables aisladas.

Muestra 1: Promocion_Croissant <= 0.50 ∧ Es_Fin_Semana > 0.50 ∧ Temperatura_Max_Prevista <= 18.00  
Muestra 2: Promocion_Croissant > 0.50 ∧ Temperatura_Max_Prevista > 15.95 ∧ Es_Fin_Semana <= 0.50 ∧ Es_Feriado <= 0.50  

|--- Promocion_Croissant >  0.50  
|   |--- Temperatura_Max_Prevista >  15.95  
|   |   |--- Es_Fin_Semana <= 0.50  
|   |   |   |--- Es_Feriado <= 0.50  
|   |   |   |   |--- value: [161.56]  

Muestra 3: Promocion_Croissant > 0.50 ∧ Temperatura_Max_Prevista > 15.95 ∧ Es_Fin_Semana > 0.50  
Muestra 4: Promocion_Croissant > 0.50 ∧ Temperatura_Max_Prevista > 15.95 ∧ Es_Fin_Semana <= 0.50 ∧ Es_Feriado <= 0.50  
Muestra 5: Promocion_Croissant <= 0.50 ∧ Es_Fin_Semana <= 0.50 ∧ Temperatura_Max_Prevista > 18.95 ∧ Es_Feriado <= 0.50  

… A partir de los splits del árbol, se reconstruyen los caminos secuenciales de decisiones que explican cada predicción. Esto permite identificar qué combinaciones de variables **disparan mayores niveles de ventas estimadas**.

#### Qué significa:

Este análisis muestra las reglas específicas que activan cada predicción. Cada observación se clasifica siguiendo un camino secuencial de decisiones.  
Permite seleccionar y priorizar combinaciones de condiciones con alto impacto esperado en ventas, según la lógica del modelo entrenado.

#### Interpretación:
- Permite rastrear por qué una predicción dio el resultado que dio.
- Identifica combinaciones reales de condiciones, no solo variables individuales.
- Identifica combinaciones específicas de alto impacto. 
---


### 4. SHAP values

(Salida visual esperada: summary_plot de SHAP)

#### Qué significa:

SHAP (SHapley Additive exPlanations) cuantifica la contribución de cada variable a cada predicción individual.
- En el gráfico summary_plot, el eje X muestra el impacto (positivo o negativo) de cada variable sobre la predicción.
- Los colores muestran si el valor de esa variable era alto o bajo.

#### Interpretación esperada:
- Promoción debería aparecer con la mayor dispersión de impacto en el eje horizontal.
- El impacto de Es_Fin_Semana, Temperatura_Max_Prevista, y Es_Feriado debería verse también, con contribuciones claramente diferenciadas.
- Las variables que no aportaron nada no deberían figurar con dispersión en el gráfico (concentradas en cero).



In [ ]:
# Calculemos los escenarios de decisión


tree_ = tree_model.tree_
feature_names_array = np.array(features)

nodos_pendientes = [(0, [])]  # nodo raíz, sin condiciones
paths = []
values = []
samples = []

while nodos_pendientes:
    nodo_actual, condiciones_actuales = nodos_pendientes.pop()

    if tree_.feature[nodo_actual] != _tree.TREE_UNDEFINED:
        idx_feature = tree_.feature[nodo_actual]
        umbral = tree_.threshold[nodo_actual]
        nombre_feature = feature_names_array[idx_feature]

        # Rama izquierda: <=
        cond_izq = condiciones_actuales + [f"{nombre_feature} <= {umbral:.4f}"]
        nodos_pendientes.append((tree_.children_left[nodo_actual], cond_izq))

        # Rama derecha: >
        cond_der = condiciones_actuales + [f"{nombre_feature} > {umbral:.4f}"]
        nodos_pendientes.append((tree_.children_right[nodo_actual], cond_der))

    else:
        # Nodo hoja
        paths.append(condiciones_actuales)
        values.append(tree_.value[nodo_actual][0][0])
        samples.append(tree_.n_node_samples[nodo_actual])

# Convertimos a DataFrame
escenarios = pd.DataFrame({
    'value': values,
    'samples': samples,
    'condiciones': paths
}).sort_values(by='value', ascending=False).reset_index(drop=True)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)  # o poné un número alto como 2000 si estás en VS Code
display(escenarios)
print(escenarios)

In [ ]:
# Vistazo rápido a los escenarios: solo filtro 
qtiles = escenarios['value'].quantile([0.33, 0.66]).values
condiciones_clasificadas = []

for i, row in escenarios.iterrows():
    if row['value'] <= qtiles[0]:
        condiciones_clasificadas.append('ventas bajas')
    elif row['value'] <= qtiles[1]:
        condiciones_clasificadas.append('ventas medias')
    else:
        condiciones_clasificadas.append('ventas altas')

escenarios['categoria'] = condiciones_clasificadas

# Agrupamos y mostramos el escenario promedio más representativo de cada clase
resumen_escenarios = escenarios.groupby('categoria').apply(
    lambda g: g.loc[g['samples'].idxmax()]
).reset_index(drop=True)

pd.set_option('display.max_colwidth', None)
display(resumen_escenarios)

Hay una posibilidad más robusta? qué podríamos hacer para mejorar esto? 


- vamos a dividir en terciles los resultados del arbol para clasificar los escenarios de ventas en máximo, medio, mínimo, y vamos a tomar los valores medios de cada tercil. 

In [ ]:
# Definición de los escenarios: 

# Primero clasificamos los nodos en categorías
terciles = escenarios['value'].quantile([1/3, 2/3])
umbral_bajo = terciles.iloc[0]
umbral_medio = terciles.iloc[1]

cond_min = escenarios['value'] <= umbral_bajo
cond_med = (escenarios['value'] > umbral_bajo) & (escenarios['value'] <= umbral_medio)
cond_max = escenarios['value'] > umbral_medio

escenarios['categoria'] = None
escenarios.loc[cond_max, 'categoria'] = 'alta'
escenarios.loc[cond_med, 'categoria'] = 'media'
escenarios.loc[cond_min, 'categoria'] = 'baja'

# Para cada categoría, cálculo manual del promedio ponderado, min, max, desvío
escenarios_alta = escenarios[escenarios['categoria'] == 'alta']
total_samples_alta = escenarios_alta['samples'].sum()
media_pond_alta = (escenarios_alta['value'] * escenarios_alta['samples']).sum() / total_samples_alta
min_alta = escenarios_alta['value'].min()
max_alta = escenarios_alta['value'].max()
desvio_pond_alta = np.sqrt(((escenarios_alta['value'] - media_pond_alta) ** 2 * escenarios_alta['samples']).sum() / total_samples_alta)

escenarios_media = escenarios[escenarios['categoria'] == 'media']
total_samples_media = escenarios_media['samples'].sum()
media_pond_media = (escenarios_media['value'] * escenarios_media['samples']).sum() / total_samples_media
min_media = escenarios_media['value'].min()
max_media = escenarios_media['value'].max()
desvio_pond_media = np.sqrt(((escenarios_media['value'] - media_pond_media) ** 2 * escenarios_media['samples']).sum() / total_samples_media)

escenarios_baja = escenarios[escenarios['categoria'] == 'baja']
total_samples_baja = escenarios_baja['samples'].sum()
media_pond_baja = (escenarios_baja['value'] * escenarios_baja['samples']).sum() / total_samples_baja
min_baja = escenarios_baja['value'].min()
max_baja = escenarios_baja['value'].max()
desvio_pond_baja = np.sqrt(((escenarios_baja['value'] - media_pond_baja) ** 2 * escenarios_baja['samples']).sum() / total_samples_baja)

# Construcción de tabla resumen
resumen_escenarios = pd.DataFrame([
    {
        'categoria': 'alta',
        'valor_esperado': media_pond_alta,
        'min': min_alta,
        'max': max_alta,
        'desvio_ponderado': desvio_pond_alta,
        'nodos': len(escenarios_alta),
        'samples_total': total_samples_alta
    },
    {
        'categoria': 'media',
        'valor_esperado': media_pond_media,
        'min': min_media,
        'max': max_media,
        'desvio_ponderado': desvio_pond_media,
        'nodos': len(escenarios_media),
        'samples_total': total_samples_media
    },
    {
        'categoria': 'baja',
        'valor_esperado': media_pond_baja,
        'min': min_baja,
        'max': max_baja,
        'desvio_ponderado': desvio_pond_baja,
        'nodos': len(escenarios_baja),
        'samples_total': total_samples_baja
    }
])

display(resumen_escenarios)
print(resumen_escenarios)


In [ ]:

# Creamos columna auxiliar de categoría ya existente
escenarios['categoria'] = pd.cut(
    escenarios['value'],
    bins=[-float('inf'), escenarios['value'].quantile(1/3), escenarios['value'].quantile(2/3), float('inf')],
    labels=['baja', 'media', 'alta']
)

# Aplanamos condiciones por categoría
paths_por_categoria = {'alta': [], 'media': [], 'baja': []}

for _, row in escenarios.iterrows():
    cat = row['categoria']
    paths_por_categoria[cat] += row['condiciones']
    


for categoria, lista_de_condiciones in paths_por_categoria.items():
    print(f"\n--- Condiciones frecuentes en categoría: {categoria} ---")
    conteo = Counter(lista_de_condiciones)
    for condicion, frecuencia in conteo.most_common(10):  # top 10
        print(f"{condicion} → {frecuencia} veces")


for categoria in paths_por_categoria:
    total_nodos_cat = escenarios[escenarios['categoria'] == categoria].shape[0]
    print(f"\n--- Condiciones robustas en categoría: {categoria} ---")
    for cond, freq in Counter(paths_por_categoria[categoria]).items():
        if freq / total_nodos_cat >= 0.5:
            print(f"{cond} → {freq} / {total_nodos_cat} nodos ({round(freq / total_nodos_cat * 100)}%)")

# Random Forest

In [ ]:
print("""
                          +--------------------+
                          |   Random Forest    |
                          +--------------------+
                                   |
     +-----------------------------+-----------------------------+
     |                             |                             |
+------------+             +------------+               +------------+
|  Árbol #1  |             |  Árbol #2  |       ...     | Árbol #100 | (n_estimators)
+------------+             +------------+               +------------+
     |                             |                             |
 Predicción y₁             Predicción y₂               Predicción y₁₀₀
     \\                             |                             /
      \\___________________________ | ___________________________/
                                   ↓
                      Promedio de todas las predicciones
                                   ↓
                     → Predicción final del modelo (ŷ)
""")



In [ ]:
# Entrenamiento del Random Forest
forest_model = RandomForestRegressor(n_estimators=100, random_state=42)
forest_model.fit(X_train, y_train)

# Predicción
y_pred_rf = forest_model.predict(X_test)

# Evaluación del modelo
print("\n======== Random Forest Evaluation ========")
print("MSE del modelo:", mean_squared_error(y_test, y_pred_rf))

# nRMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
std_y_rf = np.std(y_test)
nrmse_rf = rmse_rf / std_y_rf
print("nRMSE:", round(nrmse_rf, 4))

# MASE
mae_modelo_rf = mean_absolute_error(y_test, y_pred_rf)
mae_naive_rf = mean_absolute_error(y_test, np.full_like(y_test, np.mean(y_test)))
mase_rf = mae_modelo_rf / mae_naive_rf
print("MASE:", round(mase_rf, 4))

# SMAPE
smape_rf = np.mean(
    np.abs(y_test - y_pred_rf) / ((np.abs(y_test) + np.abs(y_pred_rf)) / 2)
) * 100
print("SMAPE (%):", round(smape_rf, 2))

# R²
print("R²:", r2_score(y_test, y_pred_rf))
print("==========================================\n")

# Importancia de variables (gain total estimado por el bosque)
importancia_rf = pd.DataFrame({
    'Variable': features,
    'Importancia': forest_model.feature_importances_
}).sort_values(by='Importancia', ascending=False)

print("\n--- Importancia de variables (Random Forest) ---")
print(importancia_rf)


# Permutation Importance

perm_importance = permutation_importance(
    forest_model, X_test, y_test, n_repeats=10, random_state=42
)
importancia_perm = pd.DataFrame({
    'Variable': X_test.columns,
    'Importancia': perm_importance.importances_mean
}).sort_values(by='Importancia', ascending=False)

print("\n--- Importancia por permutación (Permutation Importance) ---")
print(importancia_perm)



# SHAP values para el Random Forest
explainer_rf = shap.TreeExplainer(forest_model)
shap_values_rf = explainer_rf.shap_values(X_test)

# Filtro de variables con impacto nulo
shap_importancia_rf = np.abs(shap_values_rf).mean(axis=0)
variables_con_impacto_rf = shap_importancia_rf > 0
X_shap_rf = X_test.loc[:, variables_con_impacto_rf]
features_shap_rf = X_shap_rf.columns.tolist()

custom_cmap_rf = LinearSegmentedColormap.from_list("gray_to_red", ["#d3d3d3", "#ff0000"])

plt.figure()
shap.summary_plot(
    shap_values_rf[:, variables_con_impacto_rf],
    X_shap_rf,
    feature_names=features_shap_rf,
    plot_size=(8, 5),
    cmap=custom_cmap_rf,
    show=False
)
plt.title('SHAP Values Impact on Model Predictions')
plt.tight_layout()
plt.show()

## Cómo se interpretan los resultados del random forest en relación al decision tree? 



Comparamos en relación a los resultados del tree. En este caso, vamos a ver que disminuyen los errores (MSE...SMAPE) y que aumenta el poder explicativo del modelo (R2)

Por otro lado, al comparar las permutations, vemos cómo baja la performance explicativa de las variable clave, pero se mantiene el orden y el rango de importancia. 


### Indicadores estructurales


| Indicador | Decision Tree | Random Forest | Mejora (Reduccción de Error) |
|-----------|---------------|---------------|-----------------|
| MSE | 234.97 | 127.44 | -45.8% |
| nRMSE | 0.4138 | 0.3048 | -26.3% |
| MASE | 0.3950 | 0.2976 | -24.7% |
| SMAPE (%) | 11.81 | 8.87 | -24.9% |
| R² | 0.8287 | 0.9071 | +7.84 pp |


### Importancia de variables por permutación

| Variable | Perm. Importance (DT) | Perm. Importance (RF) |
|----------|----------------------|----------------------|
| Promocion_Croissant | 0.9298 | 0.8836 |
| Es_Fin_Semana | 0.3939 | 0.3799 |
| Temperatura_Max_Prevista | 0.2790 | 0.2552 |
| Es_Feriado | 0.1076 | 0.1152 |
| Dia_Semana_viernes | 0.0179 | 0.0579 |
| Precio_Nuestro | 0.0073 | 0.0363 |
| Precio_Competencia | 0.0000 | 0.0070 |
| Resto de variables | 0.0000 | < 0.005 (todas) |



In [ ]:
# seguimos el próximo encuentro. 